In [4]:
{
  "cells": [
    {
      "cell_type": "code",
      "execution_count": 1,
      "metadata": {
        "colab": {
          "base_uri": "https://localhost:8080/"
        },
        "id": "s5bxZfcJrhx8",
        "outputId": "344fba03-0570-430a-9a26-5743a9b34eef"
      },
      "outputs": [
        {
          "output_type": "stream",
          "name": "stdout",
          "text": [
            "Mounted at /content/drive\n"
          ]
        }
      ],
      "source": [
        "from google.colab import drive\n",
        "drive.mount('/content/drive')"
      ],
      "id": "s5bxZfcJrhx8"
    },
    {
      "cell_type": "code",
      "execution_count": 2,
      "metadata": {
        "id": "53aaaa6a"
      },
      "outputs": [],
      "source": [
        "import pandas as pd\n",
        "import matplotlib.pyplot as plt\n",
        "import seaborn as sns\n",
        "import numpy as np\n",
        "import time\n",
        "from pandas.plotting import register_matplotlib_converters\n",
        "register_matplotlib_converters()\n",
        "import missingno as msno\n",
        "import matplotlib as mpl\n",
        "from statsmodels.tsa.seasonal import STL, seasonal_decompose\n",
        "from statsmodels.graphics.tsaplots import plot_acf, plot_pacf"
      ],
      "id": "53aaaa6a"
    },
    {
      "cell_type": "markdown",
      "metadata": {
        "id": "F9aCDTgJ_yDS"
      },
      "source": [
        "Steps taken to prepare data for modelling\n",
        "1. Load the data\n",
        "2. Handle missing values\n",
        "3. Resampling\n",
        "4. Add feature extraction\n",
        "5. One hot encoding of categorical variables\n",
        "6. Scaling of continuous variables\n",
        "7. Outlier detection and removal\n",
        "8. Subsampling\n",
        "9. Train test split\n",
        "10. Model building\n",
        "11. Model evaluation"
      ],
      "id": "F9aCDTgJ_yDS"
    },
    {
      "cell_type": "code",
      "execution_count": 3,
      "metadata": {
        "colab": {
          "base_uri": "https://localhost:8080/"
        },
        "id": "XvZvWclKKMjp",
        "outputId": "afb49e11-9314-42b5-9fc9-3003c98f7c76"
      },
      "outputs": [
        {
          "output_type": "stream",
          "name": "stderr",
          "text": [
            "/usr/local/lib/python3.10/dist-packages/sklearn/preprocessing/_encoders.py:868: FutureWarning: `sparse` was renamed to `sparse_output` in version 1.2 and will be removed in 1.4. `sparse_output` is ignored unless you leave `sparse` to its default value.\n",
            "  warnings.warn(\n",
            "<ipython-input-3-2359b95fdb43>:29: FutureWarning: The default value of numeric_only in DataFrameGroupBy.mean is deprecated. In a future version, numeric_only will default to False. Either specify numeric_only or select only columns which should be valid for the function.\n",
            "  df_daily = df_new.resample('D').mean()\n",
            "<ipython-input-3-2359b95fdb43>:30: FutureWarning: The default value of numeric_only in DataFrameGroupBy.mean is deprecated. In a future version, numeric_only will default to False. Either specify numeric_only or select only columns which should be valid for the function.\n",
            "  df_hourly = df_new.resample('H').mean()\n",
            "<ipython-input-3-2359b95fdb43>:31: FutureWarning: The default value of numeric_only in DataFrameGroupBy.mean is deprecated. In a future version, numeric_only will default to False. Either specify numeric_only or select only columns which should be valid for the function.\n",
            "  df_monthly = df_new.resample('M').mean()\n",
            "<ipython-input-3-2359b95fdb43>:32: FutureWarning: The default value of numeric_only in DataFrameGroupBy.mean is deprecated. In a future version, numeric_only will default to False. Either specify numeric_only or select only columns which should be valid for the function.\n",
            "  df_weekly = df_new.resample('W').mean()\n",
            "<ipython-input-3-2359b95fdb43>:33: FutureWarning: The default value of numeric_only in DataFrameGroupBy.mean is deprecated. In a future version, numeric_only will default to False. Either specify numeric_only or select only columns which should be valid for the function.\n",
            "  df_yearly = df_new.resample('Y').mean()\n"
          ]
        }
      ],
      "source": [
        "# Load the data\n",
        "df = pd.read_csv('/content/drive/MyDrive/household_power_consumption.txt',sep = ';',\n",
        "                parse_dates={'dt':['Date','Time']},\n",
        "                infer_datetime_format=True,\n",
        "                low_memory=False, na_values=['nan','?'],\n",
        "                index_col='dt')\n",
        "df['total_energy_consumption'] = df['Global_active_power']*1000/60 - df['Sub_metering_1'] - df['Sub_metering_2'] - df['Sub_metering_3']\n",
        "# create a copy of original data frame to do some additional exploration\n",
        "df_new = df.copy()\n",
        "\n",
        "# Handle missing values\n",
        "df_new = df_new.fillna(df_new.resample('D').mean().apply('median'))\n",
        "\n",
        "# Add feature extraction\n",
        "weekdays = {'Monday': 'Weekday', 'Tuesday': 'Weekday', 'Wednesday': 'Weekday', 'Thursday': 'Weekday', 'Friday': 'Weekday', 'Saturday': 'Weekend', 'Sunday': 'Weekend'}\n",
        "df_new['weekday'] = df.index.day_name().map(weekdays)\n",
        "seasons = {1: 'Winter', 2: 'Winter', 3: 'Spring', 4: 'Spring', 5: 'Spring', 6: 'Summer', 7: 'Summer', 8: 'Summer', 9: 'Autumn', 10: 'Autumn', 11: 'Autumn', 12: 'Winter'}\n",
        "df_new['season'] = df.index.month.map(seasons)\n",
        "df_new['year'] = df.index.year\n",
        "df_new['year'] = df_new['year'].astype('category')\n",
        "\n",
        "# One hot encoding of categorical variables\n",
        "from sklearn.preprocessing import OneHotEncoder\n",
        "encoder = OneHotEncoder(sparse=False)\n",
        "encoded = encoder.fit_transform(df_new[[\"weekday\", \"season\", \"year\"]])\n",
        "\n",
        "# Resampling\n",
        "# Assuming the data is a minutely time series, resample it to different frequencies\n",
        "df_daily = df_new.resample('D').mean()\n",
        "df_hourly = df_new.resample('H').mean()\n",
        "df_monthly = df_new.resample('M').mean()\n",
        "df_weekly = df_new.resample('W').mean()\n",
        "df_yearly = df_new.resample('Y').mean()\n",
        "\n",
        "y = df_new['total_energy_consumption']\n",
        "X = df_new.drop('total_energy_consumption', axis=1)\n",
        "X = pd.get_dummies(X)"
      ],
      "id": "XvZvWclKKMjp"
    },
    {
      "cell_type": "code",
      "source": [
        "X"
      ],
      "metadata": {
        "colab": {
          "base_uri": "https://localhost:8080/",
          "height": 629
        },
        "id": "0IkSFSiqUe6J",
        "outputId": "69d66b18-aed8-43fe-fffc-15458f1bf289"
      },
      "id": "0IkSFSiqUe6J",
      "execution_count": 4,
      "outputs": [
        {
          "output_type": "execute_result",
          "data": {
            "text/plain": [
              "                     Global_active_power  Global_reactive_power  Voltage  \\\n",
              "dt                                                                         \n",
              "2006-12-16 17:24:00                4.216                  0.418   234.84   \n",
              "2006-12-16 17:25:00                5.360                  0.436   233.63   \n",
              "2006-12-16 17:26:00                5.374                  0.498   233.29   \n",
              "2006-12-16 17:27:00                5.388                  0.502   233.74   \n",
              "2006-12-16 17:28:00                3.666                  0.528   235.68   \n",
              "...                                  ...                    ...      ...   \n",
              "2010-11-26 20:58:00                0.946                  0.000   240.43   \n",
              "2010-11-26 20:59:00                0.944                  0.000   240.00   \n",
              "2010-11-26 21:00:00                0.938                  0.000   239.82   \n",
              "2010-11-26 21:01:00                0.934                  0.000   239.70   \n",
              "2010-11-26 21:02:00                0.932                  0.000   239.55   \n",
              "\n",
              "                     Global_intensity  Sub_metering_1  Sub_metering_2  \\\n",
              "dt                                                                      \n",
              "2006-12-16 17:24:00              18.4             0.0             1.0   \n",
              "2006-12-16 17:25:00              23.0             0.0             1.0   \n",
              "2006-12-16 17:26:00              23.0             0.0             2.0   \n",
              "2006-12-16 17:27:00              23.0             0.0             1.0   \n",
              "2006-12-16 17:28:00              15.8             0.0             1.0   \n",
              "...                               ...             ...             ...   \n",
              "2010-11-26 20:58:00               4.0             0.0             0.0   \n",
              "2010-11-26 20:59:00               4.0             0.0             0.0   \n",
              "2010-11-26 21:00:00               3.8             0.0             0.0   \n",
              "2010-11-26 21:01:00               3.8             0.0             0.0   \n",
              "2010-11-26 21:02:00               3.8             0.0             0.0   \n",
              "\n",
              "                     Sub_metering_3  weekday_Weekday  weekday_Weekend  \\\n",
              "dt                                                                      \n",
              "2006-12-16 17:24:00            17.0                0                1   \n",
              "2006-12-16 17:25:00            16.0                0                1   \n",
              "2006-12-16 17:26:00            17.0                0                1   \n",
              "2006-12-16 17:27:00            17.0                0                1   \n",
              "2006-12-16 17:28:00            17.0                0                1   \n",
              "...                             ...              ...              ...   \n",
              "2010-11-26 20:58:00             0.0                1                0   \n",
              "2010-11-26 20:59:00             0.0                1                0   \n",
              "2010-11-26 21:00:00             0.0                1                0   \n",
              "2010-11-26 21:01:00             0.0                1                0   \n",
              "2010-11-26 21:02:00             0.0                1                0   \n",
              "\n",
              "                     season_Autumn  season_Spring  season_Summer  \\\n",
              "dt                                                                 \n",
              "2006-12-16 17:24:00              0              0              0   \n",
              "2006-12-16 17:25:00              0              0              0   \n",
              "2006-12-16 17:26:00              0              0              0   \n",
              "2006-12-16 17:27:00              0              0              0   \n",
              "2006-12-16 17:28:00              0              0              0   \n",
              "...                            ...            ...            ...   \n",
              "2010-11-26 20:58:00              1              0              0   \n",
              "2010-11-26 20:59:00              1              0              0   \n",
              "2010-11-26 21:00:00              1              0              0   \n",
              "2010-11-26 21:01:00              1              0              0   \n",
              "2010-11-26 21:02:00              1              0              0   \n",
              "\n",
              "                     season_Winter  year_2006  year_2007  year_2008  \\\n",
              "dt                                                                    \n",
              "2006-12-16 17:24:00              1          1          0          0   \n",
              "2006-12-16 17:25:00              1          1          0          0   \n",
              "2006-12-16 17:26:00              1          1          0          0   \n",
              "2006-12-16 17:27:00              1          1          0          0   \n",
              "2006-12-16 17:28:00              1          1          0          0   \n",
              "...                            ...        ...        ...        ...   \n",
              "2010-11-26 20:58:00              0          0          0          0   \n",
              "2010-11-26 20:59:00              0          0          0          0   \n",
              "2010-11-26 21:00:00              0          0          0          0   \n",
              "2010-11-26 21:01:00              0          0          0          0   \n",
              "2010-11-26 21:02:00              0          0          0          0   \n",
              "\n",
              "                     year_2009  year_2010  \n",
              "dt                                         \n",
              "2006-12-16 17:24:00          0          0  \n",
              "2006-12-16 17:25:00          0          0  \n",
              "2006-12-16 17:26:00          0          0  \n",
              "2006-12-16 17:27:00          0          0  \n",
              "2006-12-16 17:28:00          0          0  \n",
              "...                        ...        ...  \n",
              "2010-11-26 20:58:00          0          1  \n",
              "2010-11-26 20:59:00          0          1  \n",
              "2010-11-26 21:00:00          0          1  \n",
              "2010-11-26 21:01:00          0          1  \n",
              "2010-11-26 21:02:00          0          1  \n",
              "\n",
              "[2075259 rows x 18 columns]"
            ],
            "text/html": [
              "\n",
              "  <div id=\"df-f0fdec19-a8c0-4331-96b4-2e6076b7d625\" class=\"colab-df-container\">\n",
              "    <div>\n",
              "<style scoped>\n",
              "    .dataframe tbody tr th:only-of-type {\n",
              "        vertical-align: middle;\n",
              "    }\n",
              "\n",
              "    .dataframe tbody tr th {\n",
              "        vertical-align: top;\n",
              "    }\n",
              "\n",
              "    .dataframe thead th {\n",
              "        text-align: right;\n",
              "    }\n",
              "</style>\n",
              "<table border=\"1\" class=\"dataframe\">\n",
              "  <thead>\n",
              "    <tr style=\"text-align: right;\">\n",
              "      <th></th>\n",
              "      <th>Global_active_power</th>\n",
              "      <th>Global_reactive_power</th>\n",
              "      <th>Voltage</th>\n",
              "      <th>Global_intensity</th>\n",
              "      <th>Sub_metering_1</th>\n",
              "      <th>Sub_metering_2</th>\n",
              "      <th>Sub_metering_3</th>\n",
              "      <th>weekday_Weekday</th>\n",
              "      <th>weekday_Weekend</th>\n",
              "      <th>season_Autumn</th>\n",
              "      <th>season_Spring</th>\n",
              "      <th>season_Summer</th>\n",
              "      <th>season_Winter</th>\n",
              "      <th>year_2006</th>\n",
              "      <th>year_2007</th>\n",
              "      <th>year_2008</th>\n",
              "      <th>year_2009</th>\n",
              "      <th>year_2010</th>\n",
              "    </tr>\n",
              "    <tr>\n",
              "      <th>dt</th>\n",
              "      <th></th>\n",
              "      <th></th>\n",
              "      <th></th>\n",
              "      <th></th>\n",
              "      <th></th>\n",
              "      <th></th>\n",
              "      <th></th>\n",
              "      <th></th>\n",
              "      <th></th>\n",
              "      <th></th>\n",
              "      <th></th>\n",
              "      <th></th>\n",
              "      <th></th>\n",
              "      <th></th>\n",
              "      <th></th>\n",
              "      <th></th>\n",
              "      <th></th>\n",
              "      <th></th>\n",
              "    </tr>\n",
              "  </thead>\n",
              "  <tbody>\n",
              "    <tr>\n",
              "      <th>2006-12-16 17:24:00</th>\n",
              "      <td>4.216</td>\n",
              "      <td>0.418</td>\n",
              "      <td>234.84</td>\n",
              "      <td>18.4</td>\n",
              "      <td>0.0</td>\n",
              "      <td>1.0</td>\n",
              "      <td>17.0</td>\n",
              "      <td>0</td>\n",
              "      <td>1</td>\n",
              "      <td>0</td>\n",
              "      <td>0</td>\n",
              "      <td>0</td>\n",
              "      <td>1</td>\n",
              "      <td>1</td>\n",
              "      <td>0</td>\n",
              "      <td>0</td>\n",
              "      <td>0</td>\n",
              "      <td>0</td>\n",
              "    </tr>\n",
              "    <tr>\n",
              "      <th>2006-12-16 17:25:00</th>\n",
              "      <td>5.360</td>\n",
              "      <td>0.436</td>\n",
              "      <td>233.63</td>\n",
              "      <td>23.0</td>\n",
              "      <td>0.0</td>\n",
              "      <td>1.0</td>\n",
              "      <td>16.0</td>\n",
              "      <td>0</td>\n",
              "      <td>1</td>\n",
              "      <td>0</td>\n",
              "      <td>0</td>\n",
              "      <td>0</td>\n",
              "      <td>1</td>\n",
              "      <td>1</td>\n",
              "      <td>0</td>\n",
              "      <td>0</td>\n",
              "      <td>0</td>\n",
              "      <td>0</td>\n",
              "    </tr>\n",
              "    <tr>\n",
              "      <th>2006-12-16 17:26:00</th>\n",
              "      <td>5.374</td>\n",
              "      <td>0.498</td>\n",
              "      <td>233.29</td>\n",
              "      <td>23.0</td>\n",
              "      <td>0.0</td>\n",
              "      <td>2.0</td>\n",
              "      <td>17.0</td>\n",
              "      <td>0</td>\n",
              "      <td>1</td>\n",
              "      <td>0</td>\n",
              "      <td>0</td>\n",
              "      <td>0</td>\n",
              "      <td>1</td>\n",
              "      <td>1</td>\n",
              "      <td>0</td>\n",
              "      <td>0</td>\n",
              "      <td>0</td>\n",
              "      <td>0</td>\n",
              "    </tr>\n",
              "    <tr>\n",
              "      <th>2006-12-16 17:27:00</th>\n",
              "      <td>5.388</td>\n",
              "      <td>0.502</td>\n",
              "      <td>233.74</td>\n",
              "      <td>23.0</td>\n",
              "      <td>0.0</td>\n",
              "      <td>1.0</td>\n",
              "      <td>17.0</td>\n",
              "      <td>0</td>\n",
              "      <td>1</td>\n",
              "      <td>0</td>\n",
              "      <td>0</td>\n",
              "      <td>0</td>\n",
              "      <td>1</td>\n",
              "      <td>1</td>\n",
              "      <td>0</td>\n",
              "      <td>0</td>\n",
              "      <td>0</td>\n",
              "      <td>0</td>\n",
              "    </tr>\n",
              "    <tr>\n",
              "      <th>2006-12-16 17:28:00</th>\n",
              "      <td>3.666</td>\n",
              "      <td>0.528</td>\n",
              "      <td>235.68</td>\n",
              "      <td>15.8</td>\n",
              "      <td>0.0</td>\n",
              "      <td>1.0</td>\n",
              "      <td>17.0</td>\n",
              "      <td>0</td>\n",
              "      <td>1</td>\n",
              "      <td>0</td>\n",
              "      <td>0</td>\n",
              "      <td>0</td>\n",
              "      <td>1</td>\n",
              "      <td>1</td>\n",
              "      <td>0</td>\n",
              "      <td>0</td>\n",
              "      <td>0</td>\n",
              "      <td>0</td>\n",
              "    </tr>\n",
              "    <tr>\n",
              "      <th>...</th>\n",
              "      <td>...</td>\n",
              "      <td>...</td>\n",
              "      <td>...</td>\n",
              "      <td>...</td>\n",
              "      <td>...</td>\n",
              "      <td>...</td>\n",
              "      <td>...</td>\n",
              "      <td>...</td>\n",
              "      <td>...</td>\n",
              "      <td>...</td>\n",
              "      <td>...</td>\n",
              "      <td>...</td>\n",
              "      <td>...</td>\n",
              "      <td>...</td>\n",
              "      <td>...</td>\n",
              "      <td>...</td>\n",
              "      <td>...</td>\n",
              "      <td>...</td>\n",
              "    </tr>\n",
              "    <tr>\n",
              "      <th>2010-11-26 20:58:00</th>\n",
              "      <td>0.946</td>\n",
              "      <td>0.000</td>\n",
              "      <td>240.43</td>\n",
              "      <td>4.0</td>\n",
              "      <td>0.0</td>\n",
              "      <td>0.0</td>\n",
              "      <td>0.0</td>\n",
              "      <td>1</td>\n",
              "      <td>0</td>\n",
              "      <td>1</td>\n",
              "      <td>0</td>\n",
              "      <td>0</td>\n",
              "      <td>0</td>\n",
              "      <td>0</td>\n",
              "      <td>0</td>\n",
              "      <td>0</td>\n",
              "      <td>0</td>\n",
              "      <td>1</td>\n",
              "    </tr>\n",
              "    <tr>\n",
              "      <th>2010-11-26 20:59:00</th>\n",
              "      <td>0.944</td>\n",
              "      <td>0.000</td>\n",
              "      <td>240.00</td>\n",
              "      <td>4.0</td>\n",
              "      <td>0.0</td>\n",
              "      <td>0.0</td>\n",
              "      <td>0.0</td>\n",
              "      <td>1</td>\n",
              "      <td>0</td>\n",
              "      <td>1</td>\n",
              "      <td>0</td>\n",
              "      <td>0</td>\n",
              "      <td>0</td>\n",
              "      <td>0</td>\n",
              "      <td>0</td>\n",
              "      <td>0</td>\n",
              "      <td>0</td>\n",
              "      <td>1</td>\n",
              "    </tr>\n",
              "    <tr>\n",
              "      <th>2010-11-26 21:00:00</th>\n",
              "      <td>0.938</td>\n",
              "      <td>0.000</td>\n",
              "      <td>239.82</td>\n",
              "      <td>3.8</td>\n",
              "      <td>0.0</td>\n",
              "      <td>0.0</td>\n",
              "      <td>0.0</td>\n",
              "      <td>1</td>\n",
              "      <td>0</td>\n",
              "      <td>1</td>\n",
              "      <td>0</td>\n",
              "      <td>0</td>\n",
              "      <td>0</td>\n",
              "      <td>0</td>\n",
              "      <td>0</td>\n",
              "      <td>0</td>\n",
              "      <td>0</td>\n",
              "      <td>1</td>\n",
              "    </tr>\n",
              "    <tr>\n",
              "      <th>2010-11-26 21:01:00</th>\n",
              "      <td>0.934</td>\n",
              "      <td>0.000</td>\n",
              "      <td>239.70</td>\n",
              "      <td>3.8</td>\n",
              "      <td>0.0</td>\n",
              "      <td>0.0</td>\n",
              "      <td>0.0</td>\n",
              "      <td>1</td>\n",
              "      <td>0</td>\n",
              "      <td>1</td>\n",
              "      <td>0</td>\n",
              "      <td>0</td>\n",
              "      <td>0</td>\n",
              "      <td>0</td>\n",
              "      <td>0</td>\n",
              "      <td>0</td>\n",
              "      <td>0</td>\n",
              "      <td>1</td>\n",
              "    </tr>\n",
              "    <tr>\n",
              "      <th>2010-11-26 21:02:00</th>\n",
              "      <td>0.932</td>\n",
              "      <td>0.000</td>\n",
              "      <td>239.55</td>\n",
              "      <td>3.8</td>\n",
              "      <td>0.0</td>\n",
              "      <td>0.0</td>\n",
              "      <td>0.0</td>\n",
              "      <td>1</td>\n",
              "      <td>0</td>\n",
              "      <td>1</td>\n",
              "      <td>0</td>\n",
              "      <td>0</td>\n",
              "      <td>0</td>\n",
              "      <td>0</td>\n",
              "      <td>0</td>\n",
              "      <td>0</td>\n",
              "      <td>0</td>\n",
              "      <td>1</td>\n",
              "    </tr>\n",
              "  </tbody>\n",
              "</table>\n",
              "<p>2075259 rows × 18 columns</p>\n",
              "</div>\n",
              "    <div class=\"colab-df-buttons\">\n",
              "\n",
              "  <div class=\"colab-df-container\">\n",
              "    <button class=\"colab-df-convert\" onclick=\"convertToInteractive('df-f0fdec19-a8c0-4331-96b4-2e6076b7d625')\"\n",
              "            title=\"Convert this dataframe to an interactive table.\"\n",
              "            style=\"display:none;\">\n",
              "\n",
              "  <svg xmlns=\"http://www.w3.org/2000/svg\" height=\"24px\" viewBox=\"0 -960 960 960\">\n",
              "    <path d=\"M120-120v-720h720v720H120Zm60-500h600v-160H180v160Zm220 220h160v-160H400v160Zm0 220h160v-160H400v160ZM180-400h160v-160H180v160Zm440 0h160v-160H620v160ZM180-180h160v-160H180v160Zm440 0h160v-160H620v160Z\"/>\n",
              "  </svg>\n",
              "    </button>\n",
              "\n",
              "  <style>\n",
              "    .colab-df-container {\n",
              "      display:flex;\n",
              "      gap: 12px;\n",
              "    }\n",
              "\n",
              "    .colab-df-convert {\n",
              "      background-color: #E8F0FE;\n",
              "      border: none;\n",
              "      border-radius: 50%;\n",
              "      cursor: pointer;\n",
              "      display: none;\n",
              "      fill: #1967D2;\n",
              "      height: 32px;\n",
              "      padding: 0 0 0 0;\n",
              "      width: 32px;\n",
              "    }\n",
              "\n",
              "    .colab-df-convert:hover {\n",
              "      background-color: #E2EBFA;\n",
              "      box-shadow: 0px 1px 2px rgba(60, 64, 67, 0.3), 0px 1px 3px 1px rgba(60, 64, 67, 0.15);\n",
              "      fill: #174EA6;\n",
              "    }\n",
              "\n",
              "    .colab-df-buttons div {\n",
              "      margin-bottom: 4px;\n",
              "    }\n",
              "\n",
              "    [theme=dark] .colab-df-convert {\n",
              "      background-color: #3B4455;\n",
              "      fill: #D2E3FC;\n",
              "    }\n",
              "\n",
              "    [theme=dark] .colab-df-convert:hover {\n",
              "      background-color: #434B5C;\n",
              "      box-shadow: 0px 1px 3px 1px rgba(0, 0, 0, 0.15);\n",
              "      filter: drop-shadow(0px 1px 2px rgba(0, 0, 0, 0.3));\n",
              "      fill: #FFFFFF;\n",
              "    }\n",
              "  </style>\n",
              "\n",
              "    <script>\n",
              "      const buttonEl =\n",
              "        document.querySelector('#df-f0fdec19-a8c0-4331-96b4-2e6076b7d625 button.colab-df-convert');\n",
              "      buttonEl.style.display =\n",
              "        google.colab.kernel.accessAllowed ? 'block' : 'none';\n",
              "\n",
              "      async function convertToInteractive(key) {\n",
              "        const element = document.querySelector('#df-f0fdec19-a8c0-4331-96b4-2e6076b7d625');\n",
              "        const dataTable =\n",
              "          await google.colab.kernel.invokeFunction('convertToInteractive',\n",
              "                                                    [key], {});\n",
              "        if (!dataTable) return;\n",
              "\n",
              "        const docLinkHtml = 'Like what you see? Visit the ' +\n",
              "          '<a target=\"_blank\" href=https://colab.research.google.com/notebooks/data_table.ipynb>data table notebook</a>'\n",
              "          + ' to learn more about interactive tables.';\n",
              "        element.innerHTML = '';\n",
              "        dataTable['output_type'] = 'display_data';\n",
              "        await google.colab.output.renderOutput(dataTable, element);\n",
              "        const docLink = document.createElement('div');\n",
              "        docLink.innerHTML = docLinkHtml;\n",
              "        element.appendChild(docLink);\n",
              "      }\n",
              "    </script>\n",
              "  </div>\n",
              "\n",
              "\n",
              "<div id=\"df-574ff913-de1c-4bfa-8528-128fed7a32c8\">\n",
              "  <button class=\"colab-df-quickchart\" onclick=\"quickchart('df-574ff913-de1c-4bfa-8528-128fed7a32c8')\"\n",
              "            title=\"Suggest charts\"\n",
              "            style=\"display:none;\">\n",
              "\n",
              "<svg xmlns=\"http://www.w3.org/2000/svg\" height=\"24px\"viewBox=\"0 0 24 24\"\n",
              "     width=\"24px\">\n",
              "    <g>\n",
              "        <path d=\"M19 3H5c-1.1 0-2 .9-2 2v14c0 1.1.9 2 2 2h14c1.1 0 2-.9 2-2V5c0-1.1-.9-2-2-2zM9 17H7v-7h2v7zm4 0h-2V7h2v10zm4 0h-2v-4h2v4z\"/>\n",
              "    </g>\n",
              "</svg>\n",
              "  </button>\n",
              "\n",
              "<style>\n",
              "  .colab-df-quickchart {\n",
              "      --bg-color: #E8F0FE;\n",
              "      --fill-color: #1967D2;\n",
              "      --hover-bg-color: #E2EBFA;\n",
              "      --hover-fill-color: #174EA6;\n",
              "      --disabled-fill-color: #AAA;\n",
              "      --disabled-bg-color: #DDD;\n",
              "  }\n",
              "\n",
              "  [theme=dark] .colab-df-quickchart {\n",
              "      --bg-color: #3B4455;\n",
              "      --fill-color: #D2E3FC;\n",
              "      --hover-bg-color: #434B5C;\n",
              "      --hover-fill-color: #FFFFFF;\n",
              "      --disabled-bg-color: #3B4455;\n",
              "      --disabled-fill-color: #666;\n",
              "  }\n",
              "\n",
              "  .colab-df-quickchart {\n",
              "    background-color: var(--bg-color);\n",
              "    border: none;\n",
              "    border-radius: 50%;\n",
              "    cursor: pointer;\n",
              "    display: none;\n",
              "    fill: var(--fill-color);\n",
              "    height: 32px;\n",
              "    padding: 0;\n",
              "    width: 32px;\n",
              "  }\n",
              "\n",
              "  .colab-df-quickchart:hover {\n",
              "    background-color: var(--hover-bg-color);\n",
              "    box-shadow: 0 1px 2px rgba(60, 64, 67, 0.3), 0 1px 3px 1px rgba(60, 64, 67, 0.15);\n",
              "    fill: var(--button-hover-fill-color);\n",
              "  }\n",
              "\n",
              "  .colab-df-quickchart-complete:disabled,\n",
              "  .colab-df-quickchart-complete:disabled:hover {\n",
              "    background-color: var(--disabled-bg-color);\n",
              "    fill: var(--disabled-fill-color);\n",
              "    box-shadow: none;\n",
              "  }\n",
              "\n",
              "  .colab-df-spinner {\n",
              "    border: 2px solid var(--fill-color);\n",
              "    border-color: transparent;\n",
              "    border-bottom-color: var(--fill-color);\n",
              "    animation:\n",
              "      spin 1s steps(1) infinite;\n",
              "  }\n",
              "\n",
              "  @keyframes spin {\n",
              "    0% {\n",
              "      border-color: transparent;\n",
              "      border-bottom-color: var(--fill-color);\n",
              "      border-left-color: var(--fill-color);\n",
              "    }\n",
              "    20% {\n",
              "      border-color: transparent;\n",
              "      border-left-color: var(--fill-color);\n",
              "      border-top-color: var(--fill-color);\n",
              "    }\n",
              "    30% {\n",
              "      border-color: transparent;\n",
              "      border-left-color: var(--fill-color);\n",
              "      border-top-color: var(--fill-color);\n",
              "      border-right-color: var(--fill-color);\n",
              "    }\n",
              "    40% {\n",
              "      border-color: transparent;\n",
              "      border-right-color: var(--fill-color);\n",
              "      border-top-color: var(--fill-color);\n",
              "    }\n",
              "    60% {\n",
              "      border-color: transparent;\n",
              "      border-right-color: var(--fill-color);\n",
              "    }\n",
              "    80% {\n",
              "      border-color: transparent;\n",
              "      border-right-color: var(--fill-color);\n",
              "      border-bottom-color: var(--fill-color);\n",
              "    }\n",
              "    90% {\n",
              "      border-color: transparent;\n",
              "      border-bottom-color: var(--fill-color);\n",
              "    }\n",
              "  }\n",
              "</style>\n",
              "\n",
              "  <script>\n",
              "    async function quickchart(key) {\n",
              "      const quickchartButtonEl =\n",
              "        document.querySelector('#' + key + ' button');\n",
              "      quickchartButtonEl.disabled = true;  // To prevent multiple clicks.\n",
              "      quickchartButtonEl.classList.add('colab-df-spinner');\n",
              "      try {\n",
              "        const charts = await google.colab.kernel.invokeFunction(\n",
              "            'suggestCharts', [key], {});\n",
              "      } catch (error) {\n",
              "        console.error('Error during call to suggestCharts:', error);\n",
              "      }\n",
              "      quickchartButtonEl.classList.remove('colab-df-spinner');\n",
              "      quickchartButtonEl.classList.add('colab-df-quickchart-complete');\n",
              "    }\n",
              "    (() => {\n",
              "      let quickchartButtonEl =\n",
              "        document.querySelector('#df-574ff913-de1c-4bfa-8528-128fed7a32c8 button');\n",
              "      quickchartButtonEl.style.display =\n",
              "        google.colab.kernel.accessAllowed ? 'block' : 'none';\n",
              "    })();\n",
              "  </script>\n",
              "</div>\n",
              "    </div>\n",
              "  </div>\n"
            ]
          },
          "metadata": {},
          "execution_count": 4
        }
      ]
    },
    {
      "cell_type": "code",
      "source": [
        "# Scaling of continuous variables\n",
        "# Assuming the data has continuous variables called \"price\" and \"volume\"\n",
        "from sklearn.preprocessing import MinMaxScaler\n",
        "scaler = MinMaxScaler()\n",
        "scaled = scaler.fit_transform(X)\n",
        "y_scaled = scaler.fit_transform(y.to_numpy().reshape(-1, 1))\n",
        "y_scaled = pd.DataFrame(y_scaled)\n",
        "X_scaled = pd.DataFrame(scaled)\n",
        "\n",
        "X_scaled"
      ],
      "metadata": {
        "colab": {
          "base_uri": "https://localhost:8080/",
          "height": 424
        },
        "id": "wX-VLuRlVPSQ",
        "outputId": "760915d3-809f-4c30-aa7e-76263926d2e5"
      },
      "id": "wX-VLuRlVPSQ",
      "execution_count": 5,
      "outputs": [
        {
          "output_type": "execute_result",
          "data": {
            "text/plain": [
              "               0         1         2         3    4       5         6    7   \\\n",
              "0        0.374796  0.300719  0.376090  0.377593  0.0  0.0125  0.548387  0.0   \n",
              "1        0.478363  0.313669  0.336995  0.473029  0.0  0.0125  0.516129  0.0   \n",
              "2        0.479631  0.358273  0.326010  0.473029  0.0  0.0250  0.548387  0.0   \n",
              "3        0.480898  0.361151  0.340549  0.473029  0.0  0.0125  0.548387  0.0   \n",
              "4        0.325005  0.379856  0.403231  0.323651  0.0  0.0125  0.548387  0.0   \n",
              "...           ...       ...       ...       ...  ...     ...       ...  ...   \n",
              "2075254  0.078762  0.000000  0.556704  0.078838  0.0  0.0000  0.000000  1.0   \n",
              "2075255  0.078580  0.000000  0.542811  0.078838  0.0  0.0000  0.000000  1.0   \n",
              "2075256  0.078037  0.000000  0.536995  0.074689  0.0  0.0000  0.000000  1.0   \n",
              "2075257  0.077675  0.000000  0.533118  0.074689  0.0  0.0000  0.000000  1.0   \n",
              "2075258  0.077494  0.000000  0.528271  0.074689  0.0  0.0000  0.000000  1.0   \n",
              "\n",
              "          8    9    10   11   12   13   14   15   16   17  \n",
              "0        1.0  0.0  0.0  0.0  1.0  1.0  0.0  0.0  0.0  0.0  \n",
              "1        1.0  0.0  0.0  0.0  1.0  1.0  0.0  0.0  0.0  0.0  \n",
              "2        1.0  0.0  0.0  0.0  1.0  1.0  0.0  0.0  0.0  0.0  \n",
              "3        1.0  0.0  0.0  0.0  1.0  1.0  0.0  0.0  0.0  0.0  \n",
              "4        1.0  0.0  0.0  0.0  1.0  1.0  0.0  0.0  0.0  0.0  \n",
              "...      ...  ...  ...  ...  ...  ...  ...  ...  ...  ...  \n",
              "2075254  0.0  1.0  0.0  0.0  0.0  0.0  0.0  0.0  0.0  1.0  \n",
              "2075255  0.0  1.0  0.0  0.0  0.0  0.0  0.0  0.0  0.0  1.0  \n",
              "2075256  0.0  1.0  0.0  0.0  0.0  0.0  0.0  0.0  0.0  1.0  \n",
              "2075257  0.0  1.0  0.0  0.0  0.0  0.0  0.0  0.0  0.0  1.0  \n",
              "2075258  0.0  1.0  0.0  0.0  0.0  0.0  0.0  0.0  0.0  1.0  \n",
              "\n",
              "[2075259 rows x 18 columns]"
            ],
            "text/html": [
              "\n",
              "  <div id=\"df-d4214988-eaea-4519-bcca-b7855c430a7c\" class=\"colab-df-container\">\n",
              "    <div>\n",
              "<style scoped>\n",
              "    .dataframe tbody tr th:only-of-type {\n",
              "        vertical-align: middle;\n",
              "    }\n",
              "\n",
              "    .dataframe tbody tr th {\n",
              "        vertical-align: top;\n",
              "    }\n",
              "\n",
              "    .dataframe thead th {\n",
              "        text-align: right;\n",
              "    }\n",
              "</style>\n",
              "<table border=\"1\" class=\"dataframe\">\n",
              "  <thead>\n",
              "    <tr style=\"text-align: right;\">\n",
              "      <th></th>\n",
              "      <th>0</th>\n",
              "      <th>1</th>\n",
              "      <th>2</th>\n",
              "      <th>3</th>\n",
              "      <th>4</th>\n",
              "      <th>5</th>\n",
              "      <th>6</th>\n",
              "      <th>7</th>\n",
              "      <th>8</th>\n",
              "      <th>9</th>\n",
              "      <th>10</th>\n",
              "      <th>11</th>\n",
              "      <th>12</th>\n",
              "      <th>13</th>\n",
              "      <th>14</th>\n",
              "      <th>15</th>\n",
              "      <th>16</th>\n",
              "      <th>17</th>\n",
              "    </tr>\n",
              "  </thead>\n",
              "  <tbody>\n",
              "    <tr>\n",
              "      <th>0</th>\n",
              "      <td>0.374796</td>\n",
              "      <td>0.300719</td>\n",
              "      <td>0.376090</td>\n",
              "      <td>0.377593</td>\n",
              "      <td>0.0</td>\n",
              "      <td>0.0125</td>\n",
              "      <td>0.548387</td>\n",
              "      <td>0.0</td>\n",
              "      <td>1.0</td>\n",
              "      <td>0.0</td>\n",
              "      <td>0.0</td>\n",
              "      <td>0.0</td>\n",
              "      <td>1.0</td>\n",
              "      <td>1.0</td>\n",
              "      <td>0.0</td>\n",
              "      <td>0.0</td>\n",
              "      <td>0.0</td>\n",
              "      <td>0.0</td>\n",
              "    </tr>\n",
              "    <tr>\n",
              "      <th>1</th>\n",
              "      <td>0.478363</td>\n",
              "      <td>0.313669</td>\n",
              "      <td>0.336995</td>\n",
              "      <td>0.473029</td>\n",
              "      <td>0.0</td>\n",
              "      <td>0.0125</td>\n",
              "      <td>0.516129</td>\n",
              "      <td>0.0</td>\n",
              "      <td>1.0</td>\n",
              "      <td>0.0</td>\n",
              "      <td>0.0</td>\n",
              "      <td>0.0</td>\n",
              "      <td>1.0</td>\n",
              "      <td>1.0</td>\n",
              "      <td>0.0</td>\n",
              "      <td>0.0</td>\n",
              "      <td>0.0</td>\n",
              "      <td>0.0</td>\n",
              "    </tr>\n",
              "    <tr>\n",
              "      <th>2</th>\n",
              "      <td>0.479631</td>\n",
              "      <td>0.358273</td>\n",
              "      <td>0.326010</td>\n",
              "      <td>0.473029</td>\n",
              "      <td>0.0</td>\n",
              "      <td>0.0250</td>\n",
              "      <td>0.548387</td>\n",
              "      <td>0.0</td>\n",
              "      <td>1.0</td>\n",
              "      <td>0.0</td>\n",
              "      <td>0.0</td>\n",
              "      <td>0.0</td>\n",
              "      <td>1.0</td>\n",
              "      <td>1.0</td>\n",
              "      <td>0.0</td>\n",
              "      <td>0.0</td>\n",
              "      <td>0.0</td>\n",
              "      <td>0.0</td>\n",
              "    </tr>\n",
              "    <tr>\n",
              "      <th>3</th>\n",
              "      <td>0.480898</td>\n",
              "      <td>0.361151</td>\n",
              "      <td>0.340549</td>\n",
              "      <td>0.473029</td>\n",
              "      <td>0.0</td>\n",
              "      <td>0.0125</td>\n",
              "      <td>0.548387</td>\n",
              "      <td>0.0</td>\n",
              "      <td>1.0</td>\n",
              "      <td>0.0</td>\n",
              "      <td>0.0</td>\n",
              "      <td>0.0</td>\n",
              "      <td>1.0</td>\n",
              "      <td>1.0</td>\n",
              "      <td>0.0</td>\n",
              "      <td>0.0</td>\n",
              "      <td>0.0</td>\n",
              "      <td>0.0</td>\n",
              "    </tr>\n",
              "    <tr>\n",
              "      <th>4</th>\n",
              "      <td>0.325005</td>\n",
              "      <td>0.379856</td>\n",
              "      <td>0.403231</td>\n",
              "      <td>0.323651</td>\n",
              "      <td>0.0</td>\n",
              "      <td>0.0125</td>\n",
              "      <td>0.548387</td>\n",
              "      <td>0.0</td>\n",
              "      <td>1.0</td>\n",
              "      <td>0.0</td>\n",
              "      <td>0.0</td>\n",
              "      <td>0.0</td>\n",
              "      <td>1.0</td>\n",
              "      <td>1.0</td>\n",
              "      <td>0.0</td>\n",
              "      <td>0.0</td>\n",
              "      <td>0.0</td>\n",
              "      <td>0.0</td>\n",
              "    </tr>\n",
              "    <tr>\n",
              "      <th>...</th>\n",
              "      <td>...</td>\n",
              "      <td>...</td>\n",
              "      <td>...</td>\n",
              "      <td>...</td>\n",
              "      <td>...</td>\n",
              "      <td>...</td>\n",
              "      <td>...</td>\n",
              "      <td>...</td>\n",
              "      <td>...</td>\n",
              "      <td>...</td>\n",
              "      <td>...</td>\n",
              "      <td>...</td>\n",
              "      <td>...</td>\n",
              "      <td>...</td>\n",
              "      <td>...</td>\n",
              "      <td>...</td>\n",
              "      <td>...</td>\n",
              "      <td>...</td>\n",
              "    </tr>\n",
              "    <tr>\n",
              "      <th>2075254</th>\n",
              "      <td>0.078762</td>\n",
              "      <td>0.000000</td>\n",
              "      <td>0.556704</td>\n",
              "      <td>0.078838</td>\n",
              "      <td>0.0</td>\n",
              "      <td>0.0000</td>\n",
              "      <td>0.000000</td>\n",
              "      <td>1.0</td>\n",
              "      <td>0.0</td>\n",
              "      <td>1.0</td>\n",
              "      <td>0.0</td>\n",
              "      <td>0.0</td>\n",
              "      <td>0.0</td>\n",
              "      <td>0.0</td>\n",
              "      <td>0.0</td>\n",
              "      <td>0.0</td>\n",
              "      <td>0.0</td>\n",
              "      <td>1.0</td>\n",
              "    </tr>\n",
              "    <tr>\n",
              "      <th>2075255</th>\n",
              "      <td>0.078580</td>\n",
              "      <td>0.000000</td>\n",
              "      <td>0.542811</td>\n",
              "      <td>0.078838</td>\n",
              "      <td>0.0</td>\n",
              "      <td>0.0000</td>\n",
              "      <td>0.000000</td>\n",
              "      <td>1.0</td>\n",
              "      <td>0.0</td>\n",
              "      <td>1.0</td>\n",
              "      <td>0.0</td>\n",
              "      <td>0.0</td>\n",
              "      <td>0.0</td>\n",
              "      <td>0.0</td>\n",
              "      <td>0.0</td>\n",
              "      <td>0.0</td>\n",
              "      <td>0.0</td>\n",
              "      <td>1.0</td>\n",
              "    </tr>\n",
              "    <tr>\n",
              "      <th>2075256</th>\n",
              "      <td>0.078037</td>\n",
              "      <td>0.000000</td>\n",
              "      <td>0.536995</td>\n",
              "      <td>0.074689</td>\n",
              "      <td>0.0</td>\n",
              "      <td>0.0000</td>\n",
              "      <td>0.000000</td>\n",
              "      <td>1.0</td>\n",
              "      <td>0.0</td>\n",
              "      <td>1.0</td>\n",
              "      <td>0.0</td>\n",
              "      <td>0.0</td>\n",
              "      <td>0.0</td>\n",
              "      <td>0.0</td>\n",
              "      <td>0.0</td>\n",
              "      <td>0.0</td>\n",
              "      <td>0.0</td>\n",
              "      <td>1.0</td>\n",
              "    </tr>\n",
              "    <tr>\n",
              "      <th>2075257</th>\n",
              "      <td>0.077675</td>\n",
              "      <td>0.000000</td>\n",
              "      <td>0.533118</td>\n",
              "      <td>0.074689</td>\n",
              "      <td>0.0</td>\n",
              "      <td>0.0000</td>\n",
              "      <td>0.000000</td>\n",
              "      <td>1.0</td>\n",
              "      <td>0.0</td>\n",
              "      <td>1.0</td>\n",
              "      <td>0.0</td>\n",
              "      <td>0.0</td>\n",
              "      <td>0.0</td>\n",
              "      <td>0.0</td>\n",
              "      <td>0.0</td>\n",
              "      <td>0.0</td>\n",
              "      <td>0.0</td>\n",
              "      <td>1.0</td>\n",
              "    </tr>\n",
              "    <tr>\n",
              "      <th>2075258</th>\n",
              "      <td>0.077494</td>\n",
              "      <td>0.000000</td>\n",
              "      <td>0.528271</td>\n",
              "      <td>0.074689</td>\n",
              "      <td>0.0</td>\n",
              "      <td>0.0000</td>\n",
              "      <td>0.000000</td>\n",
              "      <td>1.0</td>\n",
              "      <td>0.0</td>\n",
              "      <td>1.0</td>\n",
              "      <td>0.0</td>\n",
              "      <td>0.0</td>\n",
              "      <td>0.0</td>\n",
              "      <td>0.0</td>\n",
              "      <td>0.0</td>\n",
              "      <td>0.0</td>\n",
              "      <td>0.0</td>\n",
              "      <td>1.0</td>\n",
              "    </tr>\n",
              "  </tbody>\n",
              "</table>\n",
              "<p>2075259 rows × 18 columns</p>\n",
              "</div>\n",
              "    <div class=\"colab-df-buttons\">\n",
              "\n",
              "  <div class=\"colab-df-container\">\n",
              "    <button class=\"colab-df-convert\" onclick=\"convertToInteractive('df-d4214988-eaea-4519-bcca-b7855c430a7c')\"\n",
              "            title=\"Convert this dataframe to an interactive table.\"\n",
              "            style=\"display:none;\">\n",
              "\n",
              "  <svg xmlns=\"http://www.w3.org/2000/svg\" height=\"24px\" viewBox=\"0 -960 960 960\">\n",
              "    <path d=\"M120-120v-720h720v720H120Zm60-500h600v-160H180v160Zm220 220h160v-160H400v160Zm0 220h160v-160H400v160ZM180-400h160v-160H180v160Zm440 0h160v-160H620v160ZM180-180h160v-160H180v160Zm440 0h160v-160H620v160Z\"/>\n",
              "  </svg>\n",
              "    </button>\n",
              "\n",
              "  <style>\n",
              "    .colab-df-container {\n",
              "      display:flex;\n",
              "      gap: 12px;\n",
              "    }\n",
              "\n",
              "    .colab-df-convert {\n",
              "      background-color: #E8F0FE;\n",
              "      border: none;\n",
              "      border-radius: 50%;\n",
              "      cursor: pointer;\n",
              "      display: none;\n",
              "      fill: #1967D2;\n",
              "      height: 32px;\n",
              "      padding: 0 0 0 0;\n",
              "      width: 32px;\n",
              "    }\n",
              "\n",
              "    .colab-df-convert:hover {\n",
              "      background-color: #E2EBFA;\n",
              "      box-shadow: 0px 1px 2px rgba(60, 64, 67, 0.3), 0px 1px 3px 1px rgba(60, 64, 67, 0.15);\n",
              "      fill: #174EA6;\n",
              "    }\n",
              "\n",
              "    .colab-df-buttons div {\n",
              "      margin-bottom: 4px;\n",
              "    }\n",
              "\n",
              "    [theme=dark] .colab-df-convert {\n",
              "      background-color: #3B4455;\n",
              "      fill: #D2E3FC;\n",
              "    }\n",
              "\n",
              "    [theme=dark] .colab-df-convert:hover {\n",
              "      background-color: #434B5C;\n",
              "      box-shadow: 0px 1px 3px 1px rgba(0, 0, 0, 0.15);\n",
              "      filter: drop-shadow(0px 1px 2px rgba(0, 0, 0, 0.3));\n",
              "      fill: #FFFFFF;\n",
              "    }\n",
              "  </style>\n",
              "\n",
              "    <script>\n",
              "      const buttonEl =\n",
              "        document.querySelector('#df-d4214988-eaea-4519-bcca-b7855c430a7c button.colab-df-convert');\n",
              "      buttonEl.style.display =\n",
              "        google.colab.kernel.accessAllowed ? 'block' : 'none';\n",
              "\n",
              "      async function convertToInteractive(key) {\n",
              "        const element = document.querySelector('#df-d4214988-eaea-4519-bcca-b7855c430a7c');\n",
              "        const dataTable =\n",
              "          await google.colab.kernel.invokeFunction('convertToInteractive',\n",
              "                                                    [key], {});\n",
              "        if (!dataTable) return;\n",
              "\n",
              "        const docLinkHtml = 'Like what you see? Visit the ' +\n",
              "          '<a target=\"_blank\" href=https://colab.research.google.com/notebooks/data_table.ipynb>data table notebook</a>'\n",
              "          + ' to learn more about interactive tables.';\n",
              "        element.innerHTML = '';\n",
              "        dataTable['output_type'] = 'display_data';\n",
              "        await google.colab.output.renderOutput(dataTable, element);\n",
              "        const docLink = document.createElement('div');\n",
              "        docLink.innerHTML = docLinkHtml;\n",
              "        element.appendChild(docLink);\n",
              "      }\n",
              "    </script>\n",
              "  </div>\n",
              "\n",
              "\n",
              "<div id=\"df-668a6c05-7180-41c1-a24f-327fe17822cc\">\n",
              "  <button class=\"colab-df-quickchart\" onclick=\"quickchart('df-668a6c05-7180-41c1-a24f-327fe17822cc')\"\n",
              "            title=\"Suggest charts\"\n",
              "            style=\"display:none;\">\n",
              "\n",
              "<svg xmlns=\"http://www.w3.org/2000/svg\" height=\"24px\"viewBox=\"0 0 24 24\"\n",
              "     width=\"24px\">\n",
              "    <g>\n",
              "        <path d=\"M19 3H5c-1.1 0-2 .9-2 2v14c0 1.1.9 2 2 2h14c1.1 0 2-.9 2-2V5c0-1.1-.9-2-2-2zM9 17H7v-7h2v7zm4 0h-2V7h2v10zm4 0h-2v-4h2v4z\"/>\n",
              "    </g>\n",
              "</svg>\n",
              "  </button>\n",
              "\n",
              "<style>\n",
              "  .colab-df-quickchart {\n",
              "      --bg-color: #E8F0FE;\n",
              "      --fill-color: #1967D2;\n",
              "      --hover-bg-color: #E2EBFA;\n",
              "      --hover-fill-color: #174EA6;\n",
              "      --disabled-fill-color: #AAA;\n",
              "      --disabled-bg-color: #DDD;\n",
              "  }\n",
              "\n",
              "  [theme=dark] .colab-df-quickchart {\n",
              "      --bg-color: #3B4455;\n",
              "      --fill-color: #D2E3FC;\n",
              "      --hover-bg-color: #434B5C;\n",
              "      --hover-fill-color: #FFFFFF;\n",
              "      --disabled-bg-color: #3B4455;\n",
              "      --disabled-fill-color: #666;\n",
              "  }\n",
              "\n",
              "  .colab-df-quickchart {\n",
              "    background-color: var(--bg-color);\n",
              "    border: none;\n",
              "    border-radius: 50%;\n",
              "    cursor: pointer;\n",
              "    display: none;\n",
              "    fill: var(--fill-color);\n",
              "    height: 32px;\n",
              "    padding: 0;\n",
              "    width: 32px;\n",
              "  }\n",
              "\n",
              "  .colab-df-quickchart:hover {\n",
              "    background-color: var(--hover-bg-color);\n",
              "    box-shadow: 0 1px 2px rgba(60, 64, 67, 0.3), 0 1px 3px 1px rgba(60, 64, 67, 0.15);\n",
              "    fill: var(--button-hover-fill-color);\n",
              "  }\n",
              "\n",
              "  .colab-df-quickchart-complete:disabled,\n",
              "  .colab-df-quickchart-complete:disabled:hover {\n",
              "    background-color: var(--disabled-bg-color);\n",
              "    fill: var(--disabled-fill-color);\n",
              "    box-shadow: none;\n",
              "  }\n",
              "\n",
              "  .colab-df-spinner {\n",
              "    border: 2px solid var(--fill-color);\n",
              "    border-color: transparent;\n",
              "    border-bottom-color: var(--fill-color);\n",
              "    animation:\n",
              "      spin 1s steps(1) infinite;\n",
              "  }\n",
              "\n",
              "  @keyframes spin {\n",
              "    0% {\n",
              "      border-color: transparent;\n",
              "      border-bottom-color: var(--fill-color);\n",
              "      border-left-color: var(--fill-color);\n",
              "    }\n",
              "    20% {\n",
              "      border-color: transparent;\n",
              "      border-left-color: var(--fill-color);\n",
              "      border-top-color: var(--fill-color);\n",
              "    }\n",
              "    30% {\n",
              "      border-color: transparent;\n",
              "      border-left-color: var(--fill-color);\n",
              "      border-top-color: var(--fill-color);\n",
              "      border-right-color: var(--fill-color);\n",
              "    }\n",
              "    40% {\n",
              "      border-color: transparent;\n",
              "      border-right-color: var(--fill-color);\n",
              "      border-top-color: var(--fill-color);\n",
              "    }\n",
              "    60% {\n",
              "      border-color: transparent;\n",
              "      border-right-color: var(--fill-color);\n",
              "    }\n",
              "    80% {\n",
              "      border-color: transparent;\n",
              "      border-right-color: var(--fill-color);\n",
              "      border-bottom-color: var(--fill-color);\n",
              "    }\n",
              "    90% {\n",
              "      border-color: transparent;\n",
              "      border-bottom-color: var(--fill-color);\n",
              "    }\n",
              "  }\n",
              "</style>\n",
              "\n",
              "  <script>\n",
              "    async function quickchart(key) {\n",
              "      const quickchartButtonEl =\n",
              "        document.querySelector('#' + key + ' button');\n",
              "      quickchartButtonEl.disabled = true;  // To prevent multiple clicks.\n",
              "      quickchartButtonEl.classList.add('colab-df-spinner');\n",
              "      try {\n",
              "        const charts = await google.colab.kernel.invokeFunction(\n",
              "            'suggestCharts', [key], {});\n",
              "      } catch (error) {\n",
              "        console.error('Error during call to suggestCharts:', error);\n",
              "      }\n",
              "      quickchartButtonEl.classList.remove('colab-df-spinner');\n",
              "      quickchartButtonEl.classList.add('colab-df-quickchart-complete');\n",
              "    }\n",
              "    (() => {\n",
              "      let quickchartButtonEl =\n",
              "        document.querySelector('#df-668a6c05-7180-41c1-a24f-327fe17822cc button');\n",
              "      quickchartButtonEl.style.display =\n",
              "        google.colab.kernel.accessAllowed ? 'block' : 'none';\n",
              "    })();\n",
              "  </script>\n",
              "</div>\n",
              "    </div>\n",
              "  </div>\n"
            ]
          },
          "metadata": {},
          "execution_count": 5
        }
      ]
    },
    {
      "cell_type": "code",
      "source": [
        "import statsmodels.api as sm\n",
        "# Define a function to detect outliers based on standard deviation\n",
        "def detect_outliers(data, threshold=7):\n",
        "    mean = np.mean(data)\n",
        "    std = np.std(data)\n",
        "    outliers = []\n",
        "    for i in range(len(data)):\n",
        "        z_score = (data[i] - mean) / std\n",
        "        if np.abs(z_score) > threshold:\n",
        "            outliers.append(i)\n",
        "    return outliers\n",
        "\n",
        "outliers_idx = detect_outliers(y_scaled.to_numpy())\n",
        "outliers = y_scaled.iloc[outliers_idx]\n",
        "plt.figure()\n",
        "plt.plot(y_scaled)\n",
        "plt.plot(outliers, 'o')\n",
        "plt.figure()\n",
        "y_scaled.iloc[outliers_idx] = np.median(y_scaled.to_numpy())\n",
        "plt.plot(y_scaled)\n",
        "plt.plot(y_scaled.iloc[outliers_idx], 'o')"
      ],
      "metadata": {
        "colab": {
          "base_uri": "https://localhost:8080/",
          "height": 896
        },
        "id": "G4DsR7xBj8l5",
        "outputId": "6ff31452-c917-48a0-ef7b-d2692fa50a01"
      },
      "id": "G4DsR7xBj8l5",
      "execution_count": 6,
      "outputs": [
        {
          "output_type": "execute_result",
          "data": {
            "text/plain": [
              "[<matplotlib.lines.Line2D at 0x7cd6d8c50400>]"
            ]
          },
          "metadata": {},
          "execution_count": 6
        },
        {
          "output_type": "display_data",
          "data": {
            "text/plain": [
              "<Figure size 640x480 with 1 Axes>"
            ],
            "image/png": "...\n"
          },
          "metadata": {}
        },
        {
          "output_type": "display_data",
          "data": {
            "text/plain": [
              "<Figure size 640x480 with 1 Axes>"
            ],
            "image/png": "...\n"
          },
          "metadata": {}
        }
      ]
    },
    {
      "cell_type": "code",
      "source": [
        "import numpy as np\n",
        "from sklearn.preprocessing import StandardScaler\n",
        "from tensorflow.keras.models import Sequential\n",
        "from tensorflow.keras.layers import LSTM, Dense, SimpleRNN\n",
        "from sklearn.metrics import mean_squared_error, r2_score\n",
        "\n",
        "# Subsampling\n",
        "# Use a sliding window approach to create subsamples of the data\n",
        "def set_window(window_size, y):\n",
        "    window_size = window_size\n",
        "    X1 = []\n",
        "    Y1 = []\n",
        "\n",
        "    series = y.to_numpy()\n",
        "\n",
        "    for t in range(len(series) - window_size):\n",
        "        x_ = series[t:t+window_size]\n",
        "        X1.append(x_)\n",
        "        y_ = series[t+window_size]\n",
        "        Y1.append(y_)\n",
        "\n",
        "    X1 = np.array(X1).reshape(-1,window_size)\n",
        "    Y1 = np.array(Y1)\n",
        "    N = len(X1)\n",
        "    return X1, Y1\n",
        "\n",
        "def implementRNN (series, scaler,sequence_length, split_ratio, num_rnn_units, dense_layer_size, num_epochs, batch_size):\n",
        "    start = time.perf_counter()\n",
        "    data = series\n",
        "    data = data.reshape(-1, 1)\n",
        "    data_normalized = scaler.fit_transform(data)\n",
        "    X, Y = [], []\n",
        "    for i in range(len(data_normalized) - sequence_length):\n",
        "        x_sequence = data_normalized[i:i + sequence_length]\n",
        "        y_sequence = data_normalized[i + sequence_length]\n",
        "        X.append(x_sequence)\n",
        "        Y.append(y_sequence)\n",
        "    X = np.array(X)\n",
        "    Y = np.array(Y)\n",
        "\n",
        "    split_index = int(split_ratio * len(X))\n",
        "    X_train, X_test = X[:split_index], X[split_index:]\n",
        "    Y_train, Y_test = Y[:split_index], Y[split_index:]\n",
        "\n",
        "    # Create the RNN model\n",
        "    model = Sequential()\n",
        "    model.add(SimpleRNN(units=num_rnn_units, input_shape=(sequence_length, 1)))\n",
        "    model.add(Dense(units=dense_layer_size))\n",
        "    model.compile(optimizer='adam', loss='mean_squared_error')\n",
        "    model.fit(X_train, Y_train, epochs=num_epochs, batch_size=batch_size, verbose=0)\n",
        "    predictions = model.predict(X_test)\n",
        "\n",
        "    # Inference using predicted values as input for the next prediction\n",
        "    validation_target = Y_test\n",
        "    validation_predictions = []\n",
        "    last_x = X_test[0]\n",
        "\n",
        "    while len(validation_predictions) < len(validation_target):\n",
        "        p = model.predict(last_x.reshape(1, sequence_length, 1), verbose=0)[0, 0]\n",
        "        validation_predictions.append(p)\n",
        "        last_x = np.roll(last_x, -1, axis=0)\n",
        "        last_x[-1] = p\n",
        "\n",
        "    validation_predictions = scaler.inverse_transform(np.array(validation_predictions).reshape(-1, 1))\n",
        "    validation_target = scaler.inverse_transform(np.array(Y_test).reshape(-1, 1))\n",
        "\n",
        "\n",
        "    mse = mean_squared_error(validation_target, validation_predictions)\n",
        "    r2 = r2_score(validation_target, validation_predictions)\n",
        "\n",
        "    end = time.perf_counter() # End the timer\n",
        "    elapsed = end - start # Calculate the elapsed time\n",
        "\n",
        "    print(f'Mean Squared Error: {mse}')\n",
        "    print(f'R-squared: {r2}')\n",
        "    print(f'Elapsed time: {round(elapsed,2)} seconds')\n",
        "\n",
        "    return validation_predictions"
      ],
      "metadata": {
        "id": "M6EXr7BkUXDg"
      },
      "id": "M6EXr7BkUXDg",
      "execution_count": null,
      "outputs": []
    },
    {
      "cell_type": "code",
      "source": [
        "series = set_window(10, y_scaled)[1]\n",
        "scaler = MinMaxScaler(feature_range=(0, 1))\n",
        "sequence_length = 10\n",
        "split_ratio = 0.9\n",
        "num_rnn_units = 128\n",
        "dense_layer_size = 128\n",
        "num_epochs = 70\n",
        "batch_size = 50\n",
        "# learning_rate = 0.001\n",
        "predictionsrnn = implementRNN (series, scaler,sequence_length, split_ratio, num_rnn_units, dense_layer_size, num_epochs, batch_size)"
      ],
      "metadata": {
        "colab": {
          "base_uri": "https://localhost:8080/",
          "height": 355
        },
        "id": "bFCfWjHTryv5",
        "outputId": "18399f1b-41f0-46fd-962c-50f101041406"
      },
      "id": "bFCfWjHTryv5",
      "execution_count": null,
      "outputs": [
        {
          "output_type": "error",
          "ename": "KeyboardInterrupt",
          "evalue": "ignored",
          "traceback": [
            "\u001b[0;31m---------------------------------------------------------------------------\u001b[0m",
            "\u001b[0;31mKeyboardInterrupt\u001b[0m                         Traceback (most recent call last)",
            "\u001b[0;32m<ipython-input-16-3581b9652d43>\u001b[0m in \u001b[0;36m<cell line: 10>\u001b[0;34m()\u001b[0m\n\u001b[1;32m      8\u001b[0m \u001b[0mbatch_size\u001b[0m \u001b[0;34m=\u001b[0m \u001b[0;36m50\u001b[0m\u001b[0;34m\u001b[0m\u001b[0;34m\u001b[0m\u001b[0m\n\u001b[1;32m      9\u001b[0m \u001b[0;31m# learning_rate = 0.001\u001b[0m\u001b[0;34m\u001b[0m\u001b[0;34m\u001b[0m\u001b[0m\n\u001b[0;32m---> 10\u001b[0;31m \u001b[0mpredictionsrnn\u001b[0m \u001b[0;34m=\u001b[0m \u001b[0mimplementRNN\u001b[0m \u001b[0;34m(\u001b[0m\u001b[0mseries\u001b[0m\u001b[0;34m,\u001b[0m \u001b[0mscaler\u001b[0m\u001b[0;34m,\u001b[0m\u001b[0msequence_length\u001b[0m\u001b[0;34m,\u001b[0m \u001b[0msplit_ratio\u001b[0m\u001b[0;34m,\u001b[0m \u001b[0mnum_rnn_units\u001b[0m\u001b[0;34m,\u001b[0m \u001b[0mdense_layer_size\u001b[0m\u001b[0;34m,\u001b[0m \u001b[0mnum_epochs\u001b[0m\u001b[0;34m,\u001b[0m \u001b[0mbatch_size\u001b[0m\u001b[0;34m)\u001b[0m\u001b[0;34m\u001b[0m\u001b[0;34m\u001b[0m\u001b[0m\n\u001b[0m",
            "\u001b[0;32m<ipython-input-15-43fd289a3a7d>\u001b[0m in \u001b[0;36mimplementRNN\u001b[0;34m(series, scaler, sequence_length, split_ratio, num_rnn_units, dense_layer_size, num_epochs, batch_size)\u001b[0m\n\u001b[1;32m     48\u001b[0m     \u001b[0mmodel\u001b[0m\u001b[0;34m.\u001b[0m\u001b[0madd\u001b[0m\u001b[0;34m(\u001b[0m\u001b[0mDense\u001b[0m\u001b[0;34m(\u001b[0m\u001b[0munits\u001b[0m\u001b[0;34m=\u001b[0m\u001b[0mdense_layer_size\u001b[0m\u001b[0;34m)\u001b[0m\u001b[0;34m)\u001b[0m\u001b[0;34m\u001b[0m\u001b[0;34m\u001b[0m\u001b[0m\n\u001b[1;32m     49\u001b[0m     \u001b[0mmodel\u001b[0m\u001b[0;34m.\u001b[0m\u001b[0mcompile\u001b[0m\u001b[0;34m(\u001b[0m\u001b[0moptimizer\u001b[0m\u001b[0;34m=\u001b[0m\u001b[0;34m'adam'\u001b[0m\u001b[0;34m,\u001b[0m \u001b[0mloss\u001b[0m\u001b[0;34m=\u001b[0m\u001b[0;34m'mean_squared_error'\u001b[0m\u001b[0;34m)\u001b[0m\u001b[0;34m\u001b[0m\u001b[0;34m\u001b[0m\u001b[0m\n\u001b[0;32m---> 50\u001b[0;31m     \u001b[0mmodel\u001b[0m\u001b[0;34m.\u001b[0m\u001b[0mfit\u001b[0m\u001b[0;34m(\u001b[0m\u001b[0mX_train\u001b[0m\u001b[0;34m,\u001b[0m \u001b[0mY_train\u001b[0m\u001b[0;34m,\u001b[0m \u001b[0mepochs\u001b[0m\u001b[0;34m=\u001b[0m\u001b[0mnum_epochs\u001b[0m\u001b[0;34m,\u001b[0m \u001b[0mbatch_size\u001b[0m\u001b[0;34m=\u001b[0m\u001b[0mbatch_size\u001b[0m\u001b[0;34m,\u001b[0m \u001b[0mverbose\u001b[0m\u001b[0;34m=\u001b[0m\u001b[0;36m0\u001b[0m\u001b[0;34m)\u001b[0m\u001b[0;34m\u001b[0m\u001b[0;34m\u001b[0m\u001b[0m\n\u001b[0m\u001b[1;32m     51\u001b[0m     \u001b[0mpredictions\u001b[0m \u001b[0;34m=\u001b[0m \u001b[0mmodel\u001b[0m\u001b[0;34m.\u001b[0m\u001b[0mpredict\u001b[0m\u001b[0;34m(\u001b[0m\u001b[0mX_test\u001b[0m\u001b[0;34m)\u001b[0m\u001b[0;34m\u001b[0m\u001b[0;34m\u001b[0m\u001b[0m\n\u001b[1;32m     52\u001b[0m \u001b[0;34m\u001b[0m\u001b[0m\n",
            "\u001b[0;32m/usr/local/lib/python3.10/dist-packages/keras/src/utils/traceback_utils.py\u001b[0m in \u001b[0;36merror_handler\u001b[0;34m(*args, **kwargs)\u001b[0m\n\u001b[1;32m     63\u001b[0m         \u001b[0mfiltered_tb\u001b[0m \u001b[0;34m=\u001b[0m \u001b[0;32mNone\u001b[0m\u001b[0;34m\u001b[0m\u001b[0;34m\u001b[0m\u001b[0m\n\u001b[1;32m     64\u001b[0m         \u001b[0;32mtry\u001b[0m\u001b[0;34m:\u001b[0m\u001b[0;34m\u001b[0m\u001b[0;34m\u001b[0m\u001b[0m\n\u001b[0;32m---> 65\u001b[0;31m             \u001b[0;32mreturn\u001b[0m \u001b[0mfn\u001b[0m\u001b[0;34m(\u001b[0m\u001b[0;34m*\u001b[0m\u001b[0margs\u001b[0m\u001b[0;34m,\u001b[0m \u001b[0;34m**\u001b[0m\u001b[0mkwargs\u001b[0m\u001b[0;34m)\u001b[0m\u001b[0;34m\u001b[0m\u001b[0;34m\u001b[0m\u001b[0m\n\u001b[0m\u001b[1;32m     66\u001b[0m         \u001b[0;32mexcept\u001b[0m \u001b[0mException\u001b[0m \u001b[0;32mas\u001b[0m \u001b[0me\u001b[0m\u001b[0;34m:\u001b[0m\u001b[0;34m\u001b[0m\u001b[0;34m\u001b[0m\u001b[0m\n\u001b[1;32m     67\u001b[0m             \u001b[0mfiltered_tb\u001b[0m \u001b[0;34m=\u001b[0m \u001b[0m_process_traceback_frames\u001b[0m\u001b[0;34m(\u001b[0m\u001b[0me\u001b[0m\u001b[0;34m.\u001b[0m\u001b[0m__traceback__\u001b[0m\u001b[0;34m)\u001b[0m\u001b[0;34m\u001b[0m\u001b[0;34m\u001b[0m\u001b[0m\n",
            "\u001b[0;32m/usr/local/lib/python3.10/dist-packages/keras/src/engine/training.py\u001b[0m in \u001b[0;36mfit\u001b[0;34m(self, x, y, batch_size, epochs, verbose, callbacks, validation_split, validation_data, shuffle, class_weight, sample_weight, initial_epoch, steps_per_epoch, validation_steps, validation_batch_size, validation_freq, max_queue_size, workers, use_multiprocessing)\u001b[0m\n\u001b[1;32m   1805\u001b[0m                         ):\n\u001b[1;32m   1806\u001b[0m                             \u001b[0mcallbacks\u001b[0m\u001b[0;34m.\u001b[0m\u001b[0mon_train_batch_begin\u001b[0m\u001b[0;34m(\u001b[0m\u001b[0mstep\u001b[0m\u001b[0;34m)\u001b[0m\u001b[0;34m\u001b[0m\u001b[0;34m\u001b[0m\u001b[0m\n\u001b[0;32m-> 1807\u001b[0;31m                             \u001b[0mtmp_logs\u001b[0m \u001b[0;34m=\u001b[0m \u001b[0mself\u001b[0m\u001b[0;34m.\u001b[0m\u001b[0mtrain_function\u001b[0m\u001b[0;34m(\u001b[0m\u001b[0miterator\u001b[0m\u001b[0;34m)\u001b[0m\u001b[0;34m\u001b[0m\u001b[0;34m\u001b[0m\u001b[0m\n\u001b[0m\u001b[1;32m   1808\u001b[0m                             \u001b[0;32mif\u001b[0m \u001b[0mdata_handler\u001b[0m\u001b[0;34m.\u001b[0m\u001b[0mshould_sync\u001b[0m\u001b[0;34m:\u001b[0m\u001b[0;34m\u001b[0m\u001b[0;34m\u001b[0m\u001b[0m\n\u001b[1;32m   1809\u001b[0m                                 \u001b[0mcontext\u001b[0m\u001b[0;34m.\u001b[0m\u001b[0masync_wait\u001b[0m\u001b[0;34m(\u001b[0m\u001b[0;34m)\u001b[0m\u001b[0;34m\u001b[0m\u001b[0;34m\u001b[0m\u001b[0m\n",
            "\u001b[0;32m/usr/local/lib/python3.10/dist-packages/tensorflow/python/util/traceback_utils.py\u001b[0m in \u001b[0;36merror_handler\u001b[0;34m(*args, **kwargs)\u001b[0m\n\u001b[1;32m    148\u001b[0m     \u001b[0mfiltered_tb\u001b[0m \u001b[0;34m=\u001b[0m \u001b[0;32mNone\u001b[0m\u001b[0;34m\u001b[0m\u001b[0;34m\u001b[0m\u001b[0m\n\u001b[1;32m    149\u001b[0m     \u001b[0;32mtry\u001b[0m\u001b[0;34m:\u001b[0m\u001b[0;34m\u001b[0m\u001b[0;34m\u001b[0m\u001b[0m\n\u001b[0;32m--> 150\u001b[0;31m       \u001b[0;32mreturn\u001b[0m \u001b[0mfn\u001b[0m\u001b[0;34m(\u001b[0m\u001b[0;34m*\u001b[0m\u001b[0margs\u001b[0m\u001b[0;34m,\u001b[0m \u001b[0;34m**\u001b[0m\u001b[0mkwargs\u001b[0m\u001b[0;34m)\u001b[0m\u001b[0;34m\u001b[0m\u001b[0;34m\u001b[0m\u001b[0m\n\u001b[0m\u001b[1;32m    151\u001b[0m     \u001b[0;32mexcept\u001b[0m \u001b[0mException\u001b[0m \u001b[0;32mas\u001b[0m \u001b[0me\u001b[0m\u001b[0;34m:\u001b[0m\u001b[0;34m\u001b[0m\u001b[0;34m\u001b[0m\u001b[0m\n\u001b[1;32m    152\u001b[0m       \u001b[0mfiltered_tb\u001b[0m \u001b[0;34m=\u001b[0m \u001b[0m_process_traceback_frames\u001b[0m\u001b[0;34m(\u001b[0m\u001b[0me\u001b[0m\u001b[0;34m.\u001b[0m\u001b[0m__traceback__\u001b[0m\u001b[0;34m)\u001b[0m\u001b[0;34m\u001b[0m\u001b[0;34m\u001b[0m\u001b[0m\n",
            "\u001b[0;32m/usr/local/lib/python3.10/dist-packages/tensorflow/python/eager/polymorphic_function/polymorphic_function.py\u001b[0m in \u001b[0;36m__call__\u001b[0;34m(self, *args, **kwds)\u001b[0m\n\u001b[1;32m    830\u001b[0m \u001b[0;34m\u001b[0m\u001b[0m\n\u001b[1;32m    831\u001b[0m       \u001b[0;32mwith\u001b[0m \u001b[0mOptionalXlaContext\u001b[0m\u001b[0;34m(\u001b[0m\u001b[0mself\u001b[0m\u001b[0;34m.\u001b[0m\u001b[0m_jit_compile\u001b[0m\u001b[0;34m)\u001b[0m\u001b[0;34m:\u001b[0m\u001b[0;34m\u001b[0m\u001b[0;34m\u001b[0m\u001b[0m\n\u001b[0;32m--> 832\u001b[0;31m         \u001b[0mresult\u001b[0m \u001b[0;34m=\u001b[0m \u001b[0mself\u001b[0m\u001b[0;34m.\u001b[0m\u001b[0m_call\u001b[0m\u001b[0;34m(\u001b[0m\u001b[0;34m*\u001b[0m\u001b[0margs\u001b[0m\u001b[0;34m,\u001b[0m \u001b[0;34m**\u001b[0m\u001b[0mkwds\u001b[0m\u001b[0;34m)\u001b[0m\u001b[0;34m\u001b[0m\u001b[0;34m\u001b[0m\u001b[0m\n\u001b[0m\u001b[1;32m    833\u001b[0m \u001b[0;34m\u001b[0m\u001b[0m\n\u001b[1;32m    834\u001b[0m       \u001b[0mnew_tracing_count\u001b[0m \u001b[0;34m=\u001b[0m \u001b[0mself\u001b[0m\u001b[0;34m.\u001b[0m\u001b[0mexperimental_get_tracing_count\u001b[0m\u001b[0;34m(\u001b[0m\u001b[0;34m)\u001b[0m\u001b[0;34m\u001b[0m\u001b[0;34m\u001b[0m\u001b[0m\n",
            "\u001b[0;32m/usr/local/lib/python3.10/dist-packages/tensorflow/python/eager/polymorphic_function/polymorphic_function.py\u001b[0m in \u001b[0;36m_call\u001b[0;34m(self, *args, **kwds)\u001b[0m\n\u001b[1;32m    866\u001b[0m       \u001b[0;31m# In this case we have created variables on the first call, so we run the\u001b[0m\u001b[0;34m\u001b[0m\u001b[0;34m\u001b[0m\u001b[0m\n\u001b[1;32m    867\u001b[0m       \u001b[0;31m# defunned version which is guaranteed to never create variables.\u001b[0m\u001b[0;34m\u001b[0m\u001b[0;34m\u001b[0m\u001b[0m\n\u001b[0;32m--> 868\u001b[0;31m       return tracing_compilation.call_function(\n\u001b[0m\u001b[1;32m    869\u001b[0m           \u001b[0margs\u001b[0m\u001b[0;34m,\u001b[0m \u001b[0mkwds\u001b[0m\u001b[0;34m,\u001b[0m \u001b[0mself\u001b[0m\u001b[0;34m.\u001b[0m\u001b[0m_no_variable_creation_config\u001b[0m\u001b[0;34m\u001b[0m\u001b[0;34m\u001b[0m\u001b[0m\n\u001b[1;32m    870\u001b[0m       )\n",
            "\u001b[0;32m/usr/local/lib/python3.10/dist-packages/tensorflow/python/eager/polymorphic_function/tracing_compilation.py\u001b[0m in \u001b[0;36mcall_function\u001b[0;34m(args, kwargs, tracing_options)\u001b[0m\n\u001b[1;32m    137\u001b[0m   \u001b[0mbound_args\u001b[0m \u001b[0;34m=\u001b[0m \u001b[0mfunction\u001b[0m\u001b[0;34m.\u001b[0m\u001b[0mfunction_type\u001b[0m\u001b[0;34m.\u001b[0m\u001b[0mbind\u001b[0m\u001b[0;34m(\u001b[0m\u001b[0;34m*\u001b[0m\u001b[0margs\u001b[0m\u001b[0;34m,\u001b[0m \u001b[0;34m**\u001b[0m\u001b[0mkwargs\u001b[0m\u001b[0;34m)\u001b[0m\u001b[0;34m\u001b[0m\u001b[0;34m\u001b[0m\u001b[0m\n\u001b[1;32m    138\u001b[0m   \u001b[0mflat_inputs\u001b[0m \u001b[0;34m=\u001b[0m \u001b[0mfunction\u001b[0m\u001b[0;34m.\u001b[0m\u001b[0mfunction_type\u001b[0m\u001b[0;34m.\u001b[0m\u001b[0munpack_inputs\u001b[0m\u001b[0;34m(\u001b[0m\u001b[0mbound_args\u001b[0m\u001b[0;34m)\u001b[0m\u001b[0;34m\u001b[0m\u001b[0;34m\u001b[0m\u001b[0m\n\u001b[0;32m--> 139\u001b[0;31m   return function._call_flat(  # pylint: disable=protected-access\n\u001b[0m\u001b[1;32m    140\u001b[0m       \u001b[0mflat_inputs\u001b[0m\u001b[0;34m,\u001b[0m \u001b[0mcaptured_inputs\u001b[0m\u001b[0;34m=\u001b[0m\u001b[0mfunction\u001b[0m\u001b[0;34m.\u001b[0m\u001b[0mcaptured_inputs\u001b[0m\u001b[0;34m\u001b[0m\u001b[0;34m\u001b[0m\u001b[0m\n\u001b[1;32m    141\u001b[0m   )\n",
            "\u001b[0;32m/usr/local/lib/python3.10/dist-packages/tensorflow/python/eager/polymorphic_function/concrete_function.py\u001b[0m in \u001b[0;36m_call_flat\u001b[0;34m(self, tensor_inputs, captured_inputs)\u001b[0m\n\u001b[1;32m   1321\u001b[0m         and executing_eagerly):\n\u001b[1;32m   1322\u001b[0m       \u001b[0;31m# No tape is watching; skip to running the function.\u001b[0m\u001b[0;34m\u001b[0m\u001b[0;34m\u001b[0m\u001b[0m\n\u001b[0;32m-> 1323\u001b[0;31m       \u001b[0;32mreturn\u001b[0m \u001b[0mself\u001b[0m\u001b[0;34m.\u001b[0m\u001b[0m_inference_function\u001b[0m\u001b[0;34m.\u001b[0m\u001b[0mcall_preflattened\u001b[0m\u001b[0;34m(\u001b[0m\u001b[0margs\u001b[0m\u001b[0;34m)\u001b[0m\u001b[0;34m\u001b[0m\u001b[0;34m\u001b[0m\u001b[0m\n\u001b[0m\u001b[1;32m   1324\u001b[0m     forward_backward = self._select_forward_and_backward_functions(\n\u001b[1;32m   1325\u001b[0m         \u001b[0margs\u001b[0m\u001b[0;34m,\u001b[0m\u001b[0;34m\u001b[0m\u001b[0;34m\u001b[0m\u001b[0m\n",
            "\u001b[0;32m/usr/local/lib/python3.10/dist-packages/tensorflow/python/eager/polymorphic_function/atomic_function.py\u001b[0m in \u001b[0;36mcall_preflattened\u001b[0;34m(self, args)\u001b[0m\n\u001b[1;32m    214\u001b[0m   \u001b[0;32mdef\u001b[0m \u001b[0mcall_preflattened\u001b[0m\u001b[0;34m(\u001b[0m\u001b[0mself\u001b[0m\u001b[0;34m,\u001b[0m \u001b[0margs\u001b[0m\u001b[0;34m:\u001b[0m \u001b[0mSequence\u001b[0m\u001b[0;34m[\u001b[0m\u001b[0mcore\u001b[0m\u001b[0;34m.\u001b[0m\u001b[0mTensor\u001b[0m\u001b[0;34m]\u001b[0m\u001b[0;34m)\u001b[0m \u001b[0;34m->\u001b[0m \u001b[0mAny\u001b[0m\u001b[0;34m:\u001b[0m\u001b[0;34m\u001b[0m\u001b[0;34m\u001b[0m\u001b[0m\n\u001b[1;32m    215\u001b[0m     \u001b[0;34m\"\"\"Calls with flattened tensor inputs and returns the structured output.\"\"\"\u001b[0m\u001b[0;34m\u001b[0m\u001b[0;34m\u001b[0m\u001b[0m\n\u001b[0;32m--> 216\u001b[0;31m     \u001b[0mflat_outputs\u001b[0m \u001b[0;34m=\u001b[0m \u001b[0mself\u001b[0m\u001b[0;34m.\u001b[0m\u001b[0mcall_flat\u001b[0m\u001b[0;34m(\u001b[0m\u001b[0;34m*\u001b[0m\u001b[0margs\u001b[0m\u001b[0;34m)\u001b[0m\u001b[0;34m\u001b[0m\u001b[0;34m\u001b[0m\u001b[0m\n\u001b[0m\u001b[1;32m    217\u001b[0m     \u001b[0;32mreturn\u001b[0m \u001b[0mself\u001b[0m\u001b[0;34m.\u001b[0m\u001b[0mfunction_type\u001b[0m\u001b[0;34m.\u001b[0m\u001b[0mpack_output\u001b[0m\u001b[0;34m(\u001b[0m\u001b[0mflat_outputs\u001b[0m\u001b[0;34m)\u001b[0m\u001b[0;34m\u001b[0m\u001b[0;34m\u001b[0m\u001b[0m\n\u001b[1;32m    218\u001b[0m \u001b[0;34m\u001b[0m\u001b[0m\n",
            "\u001b[0;32m/usr/local/lib/python3.10/dist-packages/tensorflow/python/eager/polymorphic_function/atomic_function.py\u001b[0m in \u001b[0;36mcall_flat\u001b[0;34m(self, *args)\u001b[0m\n\u001b[1;32m    249\u001b[0m         \u001b[0;32mwith\u001b[0m \u001b[0mrecord\u001b[0m\u001b[0;34m.\u001b[0m\u001b[0mstop_recording\u001b[0m\u001b[0;34m(\u001b[0m\u001b[0;34m)\u001b[0m\u001b[0;34m:\u001b[0m\u001b[0;34m\u001b[0m\u001b[0;34m\u001b[0m\u001b[0m\n\u001b[1;32m    250\u001b[0m           \u001b[0;32mif\u001b[0m \u001b[0mself\u001b[0m\u001b[0;34m.\u001b[0m\u001b[0m_bound_context\u001b[0m\u001b[0;34m.\u001b[0m\u001b[0mexecuting_eagerly\u001b[0m\u001b[0;34m(\u001b[0m\u001b[0;34m)\u001b[0m\u001b[0;34m:\u001b[0m\u001b[0;34m\u001b[0m\u001b[0;34m\u001b[0m\u001b[0m\n\u001b[0;32m--> 251\u001b[0;31m             outputs = self._bound_context.call_function(\n\u001b[0m\u001b[1;32m    252\u001b[0m                 \u001b[0mself\u001b[0m\u001b[0;34m.\u001b[0m\u001b[0mname\u001b[0m\u001b[0;34m,\u001b[0m\u001b[0;34m\u001b[0m\u001b[0;34m\u001b[0m\u001b[0m\n\u001b[1;32m    253\u001b[0m                 \u001b[0mlist\u001b[0m\u001b[0;34m(\u001b[0m\u001b[0margs\u001b[0m\u001b[0;34m)\u001b[0m\u001b[0;34m,\u001b[0m\u001b[0;34m\u001b[0m\u001b[0;34m\u001b[0m\u001b[0m\n",
            "\u001b[0;32m/usr/local/lib/python3.10/dist-packages/tensorflow/python/eager/context.py\u001b[0m in \u001b[0;36mcall_function\u001b[0;34m(self, name, tensor_inputs, num_outputs)\u001b[0m\n\u001b[1;32m   1484\u001b[0m     \u001b[0mcancellation_context\u001b[0m \u001b[0;34m=\u001b[0m \u001b[0mcancellation\u001b[0m\u001b[0;34m.\u001b[0m\u001b[0mcontext\u001b[0m\u001b[0;34m(\u001b[0m\u001b[0;34m)\u001b[0m\u001b[0;34m\u001b[0m\u001b[0;34m\u001b[0m\u001b[0m\n\u001b[1;32m   1485\u001b[0m     \u001b[0;32mif\u001b[0m \u001b[0mcancellation_context\u001b[0m \u001b[0;32mis\u001b[0m \u001b[0;32mNone\u001b[0m\u001b[0;34m:\u001b[0m\u001b[0;34m\u001b[0m\u001b[0;34m\u001b[0m\u001b[0m\n\u001b[0;32m-> 1486\u001b[0;31m       outputs = execute.execute(\n\u001b[0m\u001b[1;32m   1487\u001b[0m           \u001b[0mname\u001b[0m\u001b[0;34m.\u001b[0m\u001b[0mdecode\u001b[0m\u001b[0;34m(\u001b[0m\u001b[0;34m\"utf-8\"\u001b[0m\u001b[0;34m)\u001b[0m\u001b[0;34m,\u001b[0m\u001b[0;34m\u001b[0m\u001b[0;34m\u001b[0m\u001b[0m\n\u001b[1;32m   1488\u001b[0m           \u001b[0mnum_outputs\u001b[0m\u001b[0;34m=\u001b[0m\u001b[0mnum_outputs\u001b[0m\u001b[0;34m,\u001b[0m\u001b[0;34m\u001b[0m\u001b[0;34m\u001b[0m\u001b[0m\n",
            "\u001b[0;32m/usr/local/lib/python3.10/dist-packages/tensorflow/python/eager/execute.py\u001b[0m in \u001b[0;36mquick_execute\u001b[0;34m(op_name, num_outputs, inputs, attrs, ctx, name)\u001b[0m\n\u001b[1;32m     51\u001b[0m   \u001b[0;32mtry\u001b[0m\u001b[0;34m:\u001b[0m\u001b[0;34m\u001b[0m\u001b[0;34m\u001b[0m\u001b[0m\n\u001b[1;32m     52\u001b[0m     \u001b[0mctx\u001b[0m\u001b[0;34m.\u001b[0m\u001b[0mensure_initialized\u001b[0m\u001b[0;34m(\u001b[0m\u001b[0;34m)\u001b[0m\u001b[0;34m\u001b[0m\u001b[0;34m\u001b[0m\u001b[0m\n\u001b[0;32m---> 53\u001b[0;31m     tensors = pywrap_tfe.TFE_Py_Execute(ctx._handle, device_name, op_name,\n\u001b[0m\u001b[1;32m     54\u001b[0m                                         inputs, attrs, num_outputs)\n\u001b[1;32m     55\u001b[0m   \u001b[0;32mexcept\u001b[0m \u001b[0mcore\u001b[0m\u001b[0;34m.\u001b[0m\u001b[0m_NotOkStatusException\u001b[0m \u001b[0;32mas\u001b[0m \u001b[0me\u001b[0m\u001b[0;34m:\u001b[0m\u001b[0;34m\u001b[0m\u001b[0;34m\u001b[0m\u001b[0m\n",
            "\u001b[0;31mKeyboardInterrupt\u001b[0m: "
          ]
        }
      ]
    },
    {
      "cell_type": "code",
      "source": [
        "# # Train test split\n",
        "# # Split the data into 80% train and 20% test sets\n",
        "# from sklearn.model_selection import train_test_split\n",
        "# X_train, X_test, y_train, y_test = train_test_split(X_scaled, y_scaled, test_size=0.2, shuffle=False)\n",
        "\n",
        "# # Model building\n",
        "# # use LSTM\n",
        "# from keras.models import Sequential\n",
        "# from keras.layers import Dense, LSTM\n",
        "# from sklearn.metrics import mean_absolute_error, mean_absolute_percentage_error\n",
        "\n",
        "# # Define a function to create the LSTM model\n",
        "# def create_LSTM(hidden_units, dense_units, input_shape, activation):\n",
        "#     model = Sequential()\n",
        "#     model.add(LSTM(hidden_units, input_shape=input_shape, activation=activation[0]))\n",
        "#     model.add(Dense(units=dense_units, activation=activation[1]))\n",
        "#     model.compile(loss='mean_absolute_error', optimizer='adam', metrics=['mean_absolute_percentage_error'])\n",
        "#     return model\n",
        "\n",
        "# # Define a function to train and evaluate the LSTM model\n",
        "# def train_evaluate_LSTM(model, X_train, y_train, X_test, y_test, sequence_length, num_epochs, batch_size):\n",
        "#     # Create subsamples of the data using the sliding window approach\n",
        "#     X_train, y_train = set_window(sequence_length, y_train)\n",
        "#     X_test, y_test = set_window(sequence_length, y_test)\n",
        "#     # Reshape the data to match the input shape of the LSTM layer\n",
        "#     X_train = X_train.reshape(X_train.shape[0], X_train.shape[1], 1)\n",
        "#     X_test = X_test.reshape(X_test.shape[0], X_test.shape[1], 1)\n",
        "#     # Train the model\n",
        "#     model.fit(X_train, y_train, epochs=num_epochs, batch_size=batch_size, verbose=0)\n",
        "#     # Evaluate the model on the test set\n",
        "#     test_loss, test_mape = model.evaluate(X_test, y_test, verbose=0)\n",
        "#     # Predict the values for the test set\n",
        "#     predictions = model.predict(X_test)\n",
        "#     # Inverse transform the predictions and the actual values to the original scale\n",
        "#     predictions = scaler.inverse_transform(predictions)\n",
        "#     y_test = scaler.inverse_transform(y_test)\n",
        "#     # Calculate the mean absolute error and the mean absolute percentage error\n",
        "#     mae = mean_absolute_error(y_test, predictions)\n",
        "#     mape = mean_absolute_percentage_error(y_test, predictions)\n",
        "#     # Return the results\n",
        "#     return test_loss, test_mape, mae, mape, predictions, y_test\n",
        "\n",
        "# # Define a function to perform grid search for the optimal hyperparameters\n",
        "# def grid_search_LSTM(X_train, y_train, X_test, y_test, sequence_lengths, hidden_units, dense_units, activations, num_epochs, batch_size):\n",
        "#     # Initialize the lists to store the results\n",
        "#     test_losses = []\n",
        "#     test_mapes = []\n",
        "#     maes = []\n",
        "#     mapes = []\n",
        "#     predictions = []\n",
        "#     actuals = []\n",
        "#     # Loop over the possible combinations of the hyperparameters\n",
        "#     for sequence_length in sequence_lengths:\n",
        "#         for hidden_unit in hidden_units:\n",
        "#             for dense_unit in dense_units:\n",
        "#                 for activation in activations:\n",
        "#                     # Create the LSTM model\n",
        "#                     model = create_LSTM(hidden_unit, dense_unit, (sequence_length, 1), activation)\n",
        "#                     # Train and evaluate the LSTM model\n",
        "#                     test_loss, test_mape, mae, mape, prediction, actual = train_evaluate_LSTM(model, X_train, y_train, X_test, y_test, sequence_length, num_epochs, batch_size)\n",
        "#                     # Append the results to the lists\n",
        "#                     test_losses.append(test_loss)\n",
        "#                     test_mapes.append(test_mape)\n",
        "#                     maes.append(mae)\n",
        "#                     mapes.append(mape)\n",
        "#                     predictions.append(prediction)\n",
        "#                     actuals.append(actual)\n",
        "#     # Find the index of the minimum test loss\n",
        "#     best_index = test_losses.index(min(test_losses))\n",
        "#     # Return the best results\n",
        "#     return test_losses[best_index], test_mapes[best_index], maes[best_index], mapes[best_index], predictions[best_index], actuals[best_index]\n",
        "\n",
        "# # Define the range of the hyperparameters\n",
        "# sequence_lengths = [5, 10, 15, 20]\n",
        "# hidden_units = [32, 64, 128]\n",
        "# dense_units = [1, 2, 4]\n",
        "# activations = [['tanh', 'linear'], ['relu', 'linear'], ['sigmoid', 'linear']]\n",
        "# num_epochs = 50\n",
        "# batch_size = 32\n",
        "\n",
        "# # Perform grid search for the optimal hyperparameters\n",
        "# test_loss, test_mape, mae, mape, predictions, actuals = grid_search_LSTM(X_train, y_train, X_test, y_test, sequence_lengths, hidden_units, dense_units, activations, num_epochs, batch_size)\n",
        "\n",
        "# # Print the results\n",
        "# print(f'Minimum test loss: {test_loss}')\n",
        "# print(f'Minimum test MAPE: {test_mape}')\n",
        "# print(f'Minimum MAE: {mae}')\n",
        "# print(f'Minimum MAPE: {mape}')\n",
        "\n",
        "# # Plot the actual and predicted values of the total energy consumption\n",
        "# plt.figure(figsize=(12, 8))\n",
        "# plt.plot(actuals, label='Actual values')\n",
        "# plt.plot(predictions, label='Predicted values')\n",
        "# plt.xlabel('Time')\n",
        "# plt.ylabel('Total energy consumption')\n",
        "# plt.title('Actual vs Predicted values')\n",
        "# plt.legend()\n",
        "# plt.show()\n",
        "\n",
        "# # Plot the residuals of the predictions\n",
        "# plt.figure(figsize=(12, 8))\n",
        "# plt.plot(actuals - predictions, label='Residuals')\n",
        "# plt.xlabel('Time')\n",
        "# plt.ylabel('Residual error')\n",
        "# plt.title('Residuals of the predictions')\n",
        "# plt.legend()\n",
        "# plt.show()\n"
      ],
      "metadata": {
        "id": "ZM1aZp-nrrzC"
      },
      "id": "ZM1aZp-nrrzC",
      "execution_count": null,
      "outputs": []
    },
    {
      "cell_type": "code",
      "execution_count": null,
      "metadata": {
        "id": "a102a78b"
      },
      "outputs": [],
      "source": [
        "df = pd.read_csv('/content/drive/MyDrive/household_power_consumption.txt',sep = ';',\n",
        "                parse_dates={'dt':['Date','Time']},\n",
        "                infer_datetime_format=True,\n",
        "                low_memory=False, na_values=['nan','?'],\n",
        "                index_col='dt')"
      ],
      "id": "a102a78b"
    },
    {
      "cell_type": "markdown",
      "metadata": {
        "id": "05251b2a"
      },
      "source": [
        "we find our target variable, y to be (global_active_power*1000/60 - sub_metering_1 - sub_metering_2 - sub_metering_3)"
      ],
      "id": "05251b2a"
    },
    {
      "cell_type": "code",
      "execution_count": null,
      "metadata": {
        "id": "4d5a0da9"
      },
      "outputs": [],
      "source": [
        "df['total_energy_consumption'] = df['Global_active_power']*1000/60 - df['Sub_metering_1'] - df['Sub_metering_2'] - df['Sub_metering_3']"
      ],
      "id": "4d5a0da9"
    },
    {
      "cell_type": "code",
      "execution_count": null,
      "metadata": {
        "id": "92b1f4b1"
      },
      "outputs": [],
      "source": [
        "# create a copy of original data frame to do some additional exploration\n",
        "df_new = df.copy()"
      ],
      "id": "92b1f4b1"
    },
    {
      "cell_type": "markdown",
      "metadata": {
        "id": "2811d331"
      },
      "source": [
        "Handling missing values"
      ],
      "id": "2811d331"
    },
    {
      "cell_type": "code",
      "execution_count": null,
      "metadata": {
        "id": "ed56e4d6"
      },
      "outputs": [],
      "source": [
        "df_new = df_new.fillna(df_new.resample('D').mean().apply('median'))"
      ],
      "id": "ed56e4d6"
    },
    {
      "cell_type": "markdown",
      "metadata": {
        "id": "0e345beb"
      },
      "source": [
        "Resampling"
      ],
      "id": "0e345beb"
    },
    {
      "cell_type": "code",
      "execution_count": null,
      "metadata": {
        "id": "d5582491"
      },
      "outputs": [],
      "source": [
        "df_daily = df_new.resample('D').mean()\n",
        "df_hourly = df_new.resample('H').mean()\n",
        "df_monthly = df_new.resample('M').mean()\n",
        "df_weekly = df_new.resample('W').mean()\n",
        "df_yearly = df_new.resample('Y').mean()"
      ],
      "id": "d5582491"
    },
    {
      "cell_type": "code",
      "execution_count": null,
      "metadata": {
        "id": "5c307dd3"
      },
      "outputs": [],
      "source": [
        "weekdays = {'Monday': 'Weekday', 'Tuesday': 'Weekday', 'Wednesday': 'Weekday', 'Thursday': 'Weekday', 'Friday': 'Weekday', 'Saturday': 'Weekend', 'Sunday': 'Weekend'}\n",
        "df_new['weekday'] = df.index.day_name().map(weekdays)"
      ],
      "id": "5c307dd3"
    },
    {
      "cell_type": "code",
      "execution_count": null,
      "metadata": {
        "id": "38dc2c38"
      },
      "outputs": [],
      "source": [
        "seasons = {1: 'Winter', 2: 'Winter', 3: 'Spring', 4: 'Spring', 5: 'Spring', 6: 'Summer', 7: 'Summer', 8: 'Summer', 9: 'Autumn', 10: 'Autumn', 11: 'Autumn', 12: 'Winter'}\n",
        "df_new['season'] = df.index.month.map(seasons)\n",
        "df_new['year'] = df.index.year"
      ],
      "id": "38dc2c38"
    },
    {
      "cell_type": "code",
      "execution_count": null,
      "metadata": {
        "id": "c21e4209"
      },
      "outputs": [],
      "source": [
        "import statsmodels.api as sm\n",
        "# Define a function to detect outliers based on standard deviation\n",
        "def detect_outliers(data, threshold=3):\n",
        "    mean = np.mean(data)\n",
        "    std = np.std(data)\n",
        "    outliers = []\n",
        "    for i in range(len(data)):\n",
        "        z_score = (data[i] - mean) / std\n",
        "        if np.abs(z_score) > threshold:\n",
        "            outliers.append(i)\n",
        "    return outliers"
      ],
      "id": "c21e4209"
    },
    {
      "cell_type": "markdown",
      "metadata": {
        "id": "753936ca"
      },
      "source": [
        "# Data Pretreatment"
      ],
      "id": "753936ca"
    },
    {
      "cell_type": "markdown",
      "metadata": {
        "id": "e84c7221"
      },
      "source": [
        "Rescaling"
      ],
      "id": "e84c7221"
    },
    {
      "cell_type": "code",
      "execution_count": null,
      "metadata": {
        "id": "a3b832a3"
      },
      "outputs": [],
      "source": [
        "y = df_daily['total_energy_consumption']\n",
        "X = df_daily.drop('total_energy_consumption', axis=1)"
      ],
      "id": "a3b832a3"
    },
    {
      "cell_type": "code",
      "execution_count": null,
      "metadata": {
        "id": "0f28805d"
      },
      "outputs": [],
      "source": [
        "from sklearn.preprocessing import StandardScaler, MinMaxScaler, FunctionTransformer"
      ],
      "id": "0f28805d"
    },
    {
      "cell_type": "code",
      "execution_count": null,
      "metadata": {
        "id": "f68d5c4a"
      },
      "outputs": [],
      "source": [
        "std = StandardScaler()\n",
        "norm = MinMaxScaler()\n",
        "logts = FunctionTransformer(np.log1p)"
      ],
      "id": "f68d5c4a"
    },
    {
      "cell_type": "code",
      "execution_count": null,
      "metadata": {
        "id": "102ec144"
      },
      "outputs": [],
      "source": [
        "y_Standard = std.fit_transform(y.to_numpy().reshape(-1, 1))\n",
        "y_Standard = pd.DataFrame(y_Standard)\n",
        "X_Standard = std.fit_transform(X.to_numpy())\n",
        "X_Standard = pd.DataFrame(X_Standard)"
      ],
      "id": "102ec144"
    },
    {
      "cell_type": "code",
      "execution_count": null,
      "metadata": {
        "id": "73a8aba3"
      },
      "outputs": [],
      "source": [
        "y_norm = norm.fit_transform(y.to_numpy().reshape(-1, 1))\n",
        "y_norm = pd.DataFrame(y_norm)\n",
        "X_norm = norm.fit_transform(X.to_numpy())\n",
        "X_norm = pd.DataFrame(X_norm)"
      ],
      "id": "73a8aba3"
    },
    {
      "cell_type": "code",
      "execution_count": null,
      "metadata": {
        "id": "3c0fcb14"
      },
      "outputs": [],
      "source": [
        "y_log = logts.transform(y)\n",
        "X_log = logts.transform(X)"
      ],
      "id": "3c0fcb14"
    },
    {
      "cell_type": "markdown",
      "metadata": {
        "id": "fd4f8a71"
      },
      "source": [
        "\n",
        "Outlier detection and removal"
      ],
      "id": "fd4f8a71"
    },
    {
      "cell_type": "code",
      "execution_count": null,
      "metadata": {
        "id": "f183d2e8"
      },
      "outputs": [],
      "source": [
        "# replacing outliers with median value\n",
        "y_Standard.iloc[outliers] = y_Standard.median()\n",
        "# y_Standard.iloc[outliers]"
      ],
      "id": "f183d2e8"
    },
    {
      "cell_type": "markdown",
      "metadata": {
        "id": "5e7bd3cc"
      },
      "source": [
        "Sub Sampling (Sliding Window Approach)"
      ],
      "id": "5e7bd3cc"
    },
    {
      "cell_type": "code",
      "execution_count": null,
      "metadata": {
        "id": "e1e32ad8"
      },
      "outputs": [],
      "source": [
        "import numpy as np\n",
        "from sklearn.model_selection import TimeSeriesSplit\n",
        "\n",
        "window_size = 10\n",
        "X1 = []\n",
        "Y1 = []\n",
        "\n",
        "series = y.to_numpy()\n",
        "\n",
        "for t in range(len(series) - window_size):\n",
        "    x_ = series[t:t+window_size]\n",
        "    X1.append(x_)\n",
        "    y_ = series[t+window_size]\n",
        "    Y1.append(y_)\n",
        "\n",
        "X1 = np.array(X1).reshape(-1,window_size)\n",
        "Y1 = np.array(Y1)\n",
        "N = len(X1)\n",
        "\n",
        "# print(X1)\n",
        "# print(Y1)"
      ],
      "id": "e1e32ad8"
    },
    {
      "cell_type": "markdown",
      "metadata": {
        "id": "d1b69f50"
      },
      "source": [
        "RNN"
      ],
      "id": "d1b69f50"
    },
    {
      "cell_type": "code",
      "execution_count": null,
      "metadata": {
        "colab": {
          "base_uri": "https://localhost:8080/"
        },
        "id": "035c930e",
        "outputId": "43722b4d-566a-466f-94ea-132b79ae0fe7"
      },
      "outputs": [
        {
          "name": "stdout",
          "output_type": "stream",
          "text": [
            "\r1/9 [==>...........................] - ETA: 1s"
          ]
        }
      ],
      "source": [
        "import numpy as np\n",
        "from sklearn.preprocessing import StandardScaler\n",
        "from tensorflow.keras.models import Sequential\n",
        "from tensorflow.keras.layers import LSTM, Dense, SimpleRNN\n",
        "from sklearn.metrics import mean_squared_error, r2_score\n",
        "\n",
        "data = series\n",
        "\n",
        "data = data.reshape(-1, 1)\n",
        "\n",
        "# Normalize the data using Min-Max scaling\n",
        "scaler = StandardScaler()\n",
        "data_normalized = scaler.fit_transform(data)\n",
        "\n",
        "# Define the sequence length (number of time steps for each input sequence)\n",
        "sequence_length = 10\n",
        "\n",
        "# Prepare sequences for input (X) and output (Y)\n",
        "X, Y = [], []\n",
        "\n",
        "for i in range(len(data_normalized) - sequence_length):\n",
        "    # Input sequence\n",
        "    x_sequence = data_normalized[i:i + sequence_length]\n",
        "    # Output sequence (target)\n",
        "    y_sequence = data_normalized[i + sequence_length]\n",
        "\n",
        "    X.append(x_sequence)\n",
        "    Y.append(y_sequence)\n",
        "\n",
        "# Convert the lists to NumPy arrays\n",
        "X = np.array(X)\n",
        "Y = np.array(Y)\n",
        "\n",
        "# Split the data into training and testing sets (80% training, 20% testing)\n",
        "split_ratio = 0.8\n",
        "split_index = int(split_ratio * len(X))\n",
        "\n",
        "X_train, X_test = X[:split_index], X[split_index:]\n",
        "Y_train, Y_test = Y[:split_index], Y[split_index:]\n",
        "\n",
        "# Create the LSTM model\n",
        "model = Sequential()\n",
        "model.add(SimpleRNN(units=100, input_shape=(sequence_length, 1)))\n",
        "model.add(Dense(units=2))\n",
        "\n",
        "# Compile the model\n",
        "model.compile(optimizer='adam', loss='mean_squared_error')\n",
        "\n",
        "# Train the model on the training set\n",
        "model.fit(X_train, Y_train, epochs=100, batch_size=50, verbose=0)\n",
        "\n",
        "# Use the trained model to make predictions on the test set\n",
        "predictions = model.predict(X_test)\n",
        "\n",
        "# Inference using predicted values as input for the next prediction\n",
        "validation_target = Y_test\n",
        "validation_predictions = []\n",
        "\n",
        "last_x = X_test[0]\n",
        "\n",
        "while len(validation_predictions) < len(validation_target):\n",
        "    p = model.predict(last_x.reshape(1, sequence_length, 1), verbose=0)[0, 0]\n",
        "    validation_predictions.append(p)\n",
        "    last_x = np.roll(last_x, -1, axis=0)\n",
        "    last_x[-1] = p\n",
        "\n",
        "# Denormalize the predictions for better interpretation\n",
        "validation_predictions = scaler.inverse_transform(np.array(validation_predictions).reshape(-1, 1))\n",
        "validation_target = scaler.inverse_transform(np.array(Y_test).reshape(-1, 1))\n",
        "\n",
        "plt.plot(validation_target, label=\"targets\")\n",
        "plt.plot(validation_predictions, label=\"predictions\")\n",
        "plt.title(\"RNN Predictions\")\n",
        "plt.legend()\n",
        "plt.show\n",
        "\n",
        "# Evaluate the model\n",
        "mse = mean_squared_error(validation_target, validation_predictions)\n",
        "r2 = r2_score(validation_target, validation_predictions)\n",
        "\n",
        "print(f'Mean Squared Error: {mse}')\n",
        "print(f'R-squared: {r2}')"
      ],
      "id": "035c930e"
    },
    {
      "cell_type": "code",
      "execution_count": null,
      "metadata": {
        "id": "541e5ee5"
      },
      "outputs": [],
      "source": [
        "def set_window(window_size, y):\n",
        "    window_size = window_size\n",
        "    X1 = []\n",
        "    Y1 = []\n",
        "\n",
        "    series = y.to_numpy()\n",
        "\n",
        "    for t in range(len(series) - window_size):\n",
        "        x_ = series[t:t+window_size]\n",
        "        X1.append(x_)\n",
        "        y_ = series[t+window_size]\n",
        "        Y1.append(y_)\n",
        "\n",
        "    X1 = np.array(X1).reshape(-1,window_size)\n",
        "    Y1 = np.array(Y1)\n",
        "    N = len(X1)\n",
        "    return series\n"
      ],
      "id": "541e5ee5"
    },
    {
      "cell_type": "code",
      "execution_count": null,
      "metadata": {
        "id": "1274298b"
      },
      "outputs": [],
      "source": [
        "def implementRNN (series, scaler,sequence_length, split_ratio, num_rnn_units, dense_layer_size, num_epochs, batch_size):\n",
        "    start = time.perf_counter()\n",
        "    data = series\n",
        "    data = data.reshape(-1, 1)\n",
        "    data_normalized = scaler.fit_transform(data)\n",
        "    X, Y = [], []\n",
        "    for i in range(len(data_normalized) - sequence_length):\n",
        "        x_sequence = data_normalized[i:i + sequence_length]\n",
        "        y_sequence = data_normalized[i + sequence_length]\n",
        "        X.append(x_sequence)\n",
        "        Y.append(y_sequence)\n",
        "    X = np.array(X)\n",
        "    Y = np.array(Y)\n",
        "\n",
        "    split_index = int(split_ratio * len(X))\n",
        "    X_train, X_test = X[:split_index], X[split_index:]\n",
        "    Y_train, Y_test = Y[:split_index], Y[split_index:]\n",
        "\n",
        "    # Create the RNN model\n",
        "    model = Sequential()\n",
        "    model.add(SimpleRNN(units=num_rnn_units, input_shape=(sequence_length, 1)))\n",
        "    model.add(Dense(units=dense_layer_size))\n",
        "    model.compile(optimizer='adam', loss='mean_squared_error')\n",
        "    model.fit(X_train, Y_train, epochs=num_epochs, batch_size=batch_size, verbose=0)\n",
        "    predictions = model.predict(X_test)\n",
        "\n",
        "    # Inference using predicted values as input for the next prediction\n",
        "    validation_target = Y_test\n",
        "    validation_predictions = []\n",
        "    last_x = X_test[0]\n",
        "\n",
        "    while len(validation_predictions) < len(validation_target):\n",
        "        p = model.predict(last_x.reshape(1, sequence_length, 1), verbose=0)[0, 0]\n",
        "        validation_predictions.append(p)\n",
        "        last_x = np.roll(last_x, -1, axis=0)\n",
        "        last_x[-1] = p\n",
        "\n",
        "    validation_predictions = scaler.inverse_transform(np.array(validation_predictions).reshape(-1, 1))\n",
        "    validation_target = scaler.inverse_transform(np.array(Y_test).reshape(-1, 1))\n",
        "\n",
        "\n",
        "    mse = mean_squared_error(validation_target, validation_predictions)\n",
        "    r2 = r2_score(validation_target, validation_predictions)\n",
        "\n",
        "    end = time.perf_counter() # End the timer\n",
        "    elapsed = end - start # Calculate the elapsed time\n",
        "\n",
        "    print(f'Mean Squared Error: {mse}')\n",
        "    print(f'R-squared: {r2}')\n",
        "    print(f'Elapsed time: {round(elapsed,2)} seconds')\n",
        "\n",
        "    return validation_predictions"
      ],
      "id": "1274298b"
    },
    {
      "cell_type": "code",
      "execution_count": null,
      "metadata": {
        "colab": {
          "base_uri": "https://localhost:8080/"
        },
        "id": "02acb5de",
        "outputId": "30dded87-ccd9-41b0-888e-41e5449b9100"
      },
      "outputs": [
        {
          "name": "stdout",
          "output_type": "stream",
          "text": [
            "5/5 [==============================] - 0s 4ms/step\n",
            "Mean Squared Error: 10.8698085769092\n",
            "R-squared: -0.3403108153566434\n",
            "Elapsed time: 21.78 seconds\n"
          ]
        }
      ],
      "source": [
        "series = set_window(10, df_daily['total_energy_consumption'])\n",
        "scaler = MinMaxScaler(feature_range=(0, 1))\n",
        "sequence_length = 10\n",
        "split_ratio = 0.9\n",
        "num_rnn_units = 128\n",
        "dense_layer_size = 128\n",
        "num_epochs = 70\n",
        "batch_size = 50\n",
        "# learning_rate = 0.001\n",
        "predictionsrnn = implementRNN (series, scaler,sequence_length, split_ratio, num_rnn_units, dense_layer_size, num_epochs, batch_size)"
      ],
      "id": "02acb5de"
    },
    {
      "cell_type": "code",
      "execution_count": null,
      "metadata": {
        "colab": {
          "base_uri": "https://localhost:8080/"
        },
        "id": "z99DnMd-k6Nv",
        "outputId": "cfc91a72-ea14-4b80-a603-2810b6431a6b"
      },
      "outputs": [
        {
          "name": "stdout",
          "output_type": "stream",
          "text": [
            "9/9 [==============================] - 0s 3ms/step\n",
            "Mean Squared Error: 8.209876801768083\n",
            "R-squared: -0.46719789390092803\n",
            "Elapsed time: 44.21 seconds\n",
            "9/9 [==============================] - 0s 2ms/step\n",
            "Mean Squared Error: 27.720037296072494\n",
            "R-squared: -3.9538843665588086\n",
            "Elapsed time: 33.23 seconds\n",
            "9/9 [==============================] - 0s 3ms/step\n",
            "Mean Squared Error: 39.057506865178276\n",
            "R-squared: -5.9800184822833575\n",
            "Elapsed time: 44.78 seconds\n",
            "9/9 [==============================] - 0s 3ms/step\n",
            "Mean Squared Error: 11.133454119842357\n",
            "R-squared: -0.9896742461418429\n",
            "Elapsed time: 35.15 seconds\n",
            "9/9 [==============================] - 0s 3ms/step\n",
            "Mean Squared Error: 7.499935061484729\n",
            "R-squared: -0.3403232706530164\n",
            "Elapsed time: 32.86 seconds\n",
            "9/9 [==============================] - 0s 2ms/step\n",
            "Mean Squared Error: 44.104561694109954\n",
            "R-squared: -6.881984296656641\n",
            "Elapsed time: 34.07 seconds\n",
            "9/9 [==============================] - 0s 3ms/step\n",
            "Mean Squared Error: 13.99916567493436\n",
            "R-squared: -1.5018093316833268\n",
            "Elapsed time: 36.81 seconds\n",
            "9/9 [==============================] - 0s 3ms/step\n",
            "Mean Squared Error: 17.85007229226058\n",
            "R-squared: -2.1900099240884874\n",
            "Elapsed time: 33.72 seconds\n",
            "9/9 [==============================] - 0s 3ms/step\n",
            "Mean Squared Error: 9.260312262627515\n",
            "R-squared: -0.6549225983106088\n",
            "Elapsed time: 45.36 seconds\n",
            "9/9 [==============================] - 0s 3ms/step\n",
            "Mean Squared Error: 14.830129871036418\n",
            "R-squared: -1.6503120373713647\n",
            "Elapsed time: 31.28 seconds\n",
            "9/9 [==============================] - 0s 4ms/step\n",
            "Mean Squared Error: 13.227047111381406\n",
            "R-squared: -1.3638230064752879\n",
            "Elapsed time: 44.22 seconds\n",
            "9/9 [==============================] - 0s 3ms/step\n",
            "Mean Squared Error: 22.041189484878444\n",
            "R-squared: -2.939009996388793\n",
            "Elapsed time: 44.9 seconds\n",
            "9/9 [==============================] - 0s 3ms/step\n",
            "Mean Squared Error: 6.364143629142614\n",
            "R-squared: -0.1373444881840531\n",
            "Elapsed time: 45.19 seconds\n",
            "9/9 [==============================] - 0s 4ms/step\n",
            "Mean Squared Error: 5.6354139194849004\n",
            "R-squared: -0.007112242189500462\n",
            "Elapsed time: 32.33 seconds\n",
            "9/9 [==============================] - 0s 3ms/step\n",
            "Mean Squared Error: 9.319567284400943\n",
            "R-squared: -0.6655121412779657\n",
            "Elapsed time: 43.73 seconds\n",
            "9/9 [==============================] - 0s 3ms/step\n",
            "Mean Squared Error: 39.04861545581809\n",
            "R-squared: -5.978429486808508\n",
            "Elapsed time: 36.0 seconds\n",
            "5/5 [==============================] - 0s 3ms/step\n",
            "Mean Squared Error: 16.832760356335157\n",
            "R-squared: -1.0755775594640467\n",
            "Elapsed time: 31.4 seconds\n",
            "5/5 [==============================] - 0s 4ms/step\n",
            "Mean Squared Error: 17.383617810109385\n",
            "R-squared: -1.143501497387097\n",
            "Elapsed time: 20.63 seconds\n",
            "5/5 [==============================] - 0s 4ms/step\n",
            "Mean Squared Error: 25.087798991164444\n",
            "R-squared: -2.093471985586014\n",
            "Elapsed time: 27.74 seconds\n",
            "5/5 [==============================] - 0s 3ms/step\n",
            "Mean Squared Error: 72.27345671545406\n",
            "R-squared: -7.911738878705966\n",
            "Elapsed time: 22.69 seconds\n",
            "5/5 [==============================] - 0s 3ms/step\n",
            "Mean Squared Error: 17.502145247086858\n",
            "R-squared: -1.15811662189213\n",
            "Elapsed time: 23.06 seconds\n",
            "5/5 [==============================] - 0s 4ms/step\n",
            "Mean Squared Error: 10.928674803835582\n",
            "R-squared: -0.34756936458043297\n",
            "Elapsed time: 20.02 seconds\n",
            "5/5 [==============================] - 0s 3ms/step\n",
            "Mean Squared Error: 35.038931464577665\n",
            "R-squared: -3.320504677541198\n",
            "Elapsed time: 27.14 seconds\n",
            "5/5 [==============================] - 0s 4ms/step\n",
            "Mean Squared Error: 8.685668082139962\n",
            "R-squared: -0.07099354940070257\n",
            "Elapsed time: 21.81 seconds\n",
            "5/5 [==============================] - 0s 4ms/step\n",
            "Mean Squared Error: 109.57631457947592\n",
            "R-squared: -12.511398889745163\n",
            "Elapsed time: 26.26 seconds\n",
            "5/5 [==============================] - 0s 4ms/step\n",
            "Mean Squared Error: 9.306007728901756\n",
            "R-squared: -0.14748504709971488\n",
            "Elapsed time: 23.01 seconds\n",
            "5/5 [==============================] - 0s 3ms/step\n",
            "Mean Squared Error: 12.701593937371767\n",
            "R-squared: -0.5661806375037839\n",
            "Elapsed time: 34.84 seconds\n",
            "5/5 [==============================] - 0s 4ms/step\n",
            "Mean Squared Error: 32.74046854587594\n",
            "R-squared: -3.0370907897220167\n",
            "Elapsed time: 33.58 seconds\n",
            "5/5 [==============================] - 0s 4ms/step\n",
            "Mean Squared Error: 8.628901482963142\n",
            "R-squared: -0.06399389652831133\n",
            "Elapsed time: 24.22 seconds\n",
            "5/5 [==============================] - 0s 4ms/step\n",
            "Mean Squared Error: 31.182661506194936\n",
            "R-squared: -2.845004093001469\n",
            "Elapsed time: 22.0 seconds\n",
            "5/5 [==============================] - 0s 4ms/step\n",
            "Mean Squared Error: 44.62762679822564\n",
            "R-squared: -4.502846755593004\n",
            "Elapsed time: 30.9 seconds\n",
            "5/5 [==============================] - 0s 4ms/step\n",
            "Mean Squared Error: 31.529173785734606\n",
            "R-squared: -2.8877310787284767\n",
            "Elapsed time: 25.6 seconds\n",
            "9/9 [==============================] - 0s 3ms/step\n",
            "Mean Squared Error: 36.73887561448686\n",
            "R-squared: -5.535151769285995\n",
            "Elapsed time: 46.07 seconds\n",
            "9/9 [==============================] - 0s 3ms/step\n",
            "Mean Squared Error: 6.340588316233943\n",
            "R-squared: -0.1278708523352594\n",
            "Elapsed time: 32.35 seconds\n",
            "9/9 [==============================] - 0s 3ms/step\n",
            "Mean Squared Error: 5.978422733000293\n",
            "R-squared: -0.06344843840839953\n",
            "Elapsed time: 42.06 seconds\n",
            "9/9 [==============================] - 0s 4ms/step\n",
            "Mean Squared Error: 18.536425144199757\n",
            "R-squared: -2.297279776564181\n",
            "Elapsed time: 36.99 seconds\n",
            "9/9 [==============================] - 0s 3ms/step\n",
            "Mean Squared Error: 6.138692136195475\n",
            "R-squared: -0.0919573368527602\n",
            "Elapsed time: 41.02 seconds\n",
            "9/9 [==============================] - 0s 3ms/step\n",
            "Mean Squared Error: 5.885182828222402\n",
            "R-squared: -0.04686282117091878\n",
            "Elapsed time: 33.58 seconds\n",
            "9/9 [==============================] - 0s 3ms/step\n",
            "Mean Squared Error: 10.679698508845417\n",
            "R-squared: -0.8997165655772335\n",
            "Elapsed time: 43.03 seconds\n",
            "9/9 [==============================] - 0s 3ms/step\n",
            "Mean Squared Error: 26.19129547185807\n",
            "R-squared: -3.6589365646049385\n",
            "Elapsed time: 45.73 seconds\n",
            "9/9 [==============================] - 0s 4ms/step\n",
            "Mean Squared Error: 72.22258220627612\n",
            "R-squared: -11.847032686586505\n",
            "Elapsed time: 47.23 seconds\n",
            "9/9 [==============================] - 0s 4ms/step\n",
            "Mean Squared Error: 42.90312220806309\n",
            "R-squared: -6.631654761240381\n",
            "Elapsed time: 38.56 seconds\n",
            "9/9 [==============================] - 0s 4ms/step\n",
            "Mean Squared Error: 10.103419599504116\n",
            "R-squared: -0.7972074367322837\n",
            "Elapsed time: 52.79 seconds\n",
            "9/9 [==============================] - 0s 4ms/step\n",
            "Mean Squared Error: 39.027109383747636\n",
            "R-squared: -5.942185319322783\n",
            "Elapsed time: 41.23 seconds\n",
            "9/9 [==============================] - 0s 4ms/step\n",
            "Mean Squared Error: 7.47832968815196\n",
            "R-squared: -0.33025354411116137\n",
            "Elapsed time: 40.03 seconds\n",
            "9/9 [==============================] - 0s 3ms/step\n",
            "Mean Squared Error: 91.01981109811392\n",
            "R-squared: -15.190704521816226\n",
            "Elapsed time: 36.0 seconds\n",
            "9/9 [==============================] - 0s 4ms/step\n",
            "Mean Squared Error: 88.89929185339315\n",
            "R-squared: -14.813504216631125\n",
            "Elapsed time: 49.88 seconds\n",
            "9/9 [==============================] - 0s 4ms/step\n",
            "Mean Squared Error: 25.247730234507642\n",
            "R-squared: -3.491094138097038\n",
            "Elapsed time: 40.44 seconds\n",
            "5/5 [==============================] - 0s 4ms/step\n",
            "Mean Squared Error: 8.37459828729661\n",
            "R-squared: -0.031709357917153014\n",
            "Elapsed time: 33.9 seconds\n",
            "5/5 [==============================] - 0s 4ms/step\n",
            "Mean Squared Error: 7.065595222610003\n",
            "R-squared: 0.1295533874766157\n",
            "Elapsed time: 21.0 seconds\n",
            "5/5 [==============================] - 0s 4ms/step\n",
            "Mean Squared Error: 26.67118325427667\n",
            "R-squared: -2.285758720140736\n",
            "Elapsed time: 53.5 seconds\n",
            "5/5 [==============================] - 0s 5ms/step\n",
            "Mean Squared Error: 9.868050314460476\n",
            "R-squared: -0.2156953091433207\n",
            "Elapsed time: 25.13 seconds\n",
            "5/5 [==============================] - 0s 4ms/step\n",
            "Mean Squared Error: 18.66129281701529\n",
            "R-squared: -1.2989795772475032\n",
            "Elapsed time: 30.79 seconds\n",
            "5/5 [==============================] - 0s 4ms/step\n",
            "Mean Squared Error: 8.638321779143956\n",
            "R-squared: -0.06419879622898494\n",
            "Elapsed time: 23.26 seconds\n",
            "5/5 [==============================] - 0s 4ms/step\n",
            "Mean Squared Error: 8.558527532670208\n",
            "R-squared: -0.05436853715616552\n",
            "Elapsed time: 55.85 seconds\n",
            "5/5 [==============================] - 0s 4ms/step\n",
            "Mean Squared Error: 15.19189302403274\n",
            "R-squared: -0.8715665706791136\n",
            "Elapsed time: 27.15 seconds\n",
            "5/5 [==============================] - 0s 4ms/step\n",
            "Mean Squared Error: 6.924991297632295\n",
            "R-squared: 0.14687510013469351\n",
            "Elapsed time: 33.75 seconds\n",
            "5/5 [==============================] - 0s 5ms/step\n",
            "Mean Squared Error: 10.298591632277715\n",
            "R-squared: -0.2687358839056737\n",
            "Elapsed time: 28.27 seconds\n",
            "5/5 [==============================] - 0s 6ms/step\n",
            "Mean Squared Error: 16.214163202415786\n",
            "R-squared: -0.9975052334275434\n",
            "Elapsed time: 53.86 seconds\n",
            "5/5 [==============================] - 0s 5ms/step\n",
            "Mean Squared Error: 24.975600751733207\n",
            "R-squared: -2.07687128757597\n",
            "Elapsed time: 53.71 seconds\n",
            "5/5 [==============================] - 0s 4ms/step\n",
            "Mean Squared Error: 11.28222091938353\n",
            "R-squared: -0.3899141787222531\n",
            "Elapsed time: 32.74 seconds\n",
            "5/5 [==============================] - 0s 5ms/step\n",
            "Mean Squared Error: 8.287579890362624\n",
            "R-squared: -0.02098911900564282\n",
            "Elapsed time: 27.79 seconds\n",
            "5/5 [==============================] - 0s 6ms/step\n",
            "Mean Squared Error: 8.98521298247241\n",
            "R-squared: -0.10693408792601944\n",
            "Elapsed time: 52.25 seconds\n",
            "5/5 [==============================] - 0s 4ms/step\n",
            "Mean Squared Error: 22.273050956036357\n",
            "R-squared: -1.743930432527793\n",
            "Elapsed time: 55.27 seconds\n",
            "9/9 [==============================] - 0s 3ms/step\n",
            "Mean Squared Error: 56.094068139237656\n",
            "R-squared: -9.024644781088583\n",
            "Elapsed time: 35.31 seconds\n",
            "9/9 [==============================] - 0s 3ms/step\n",
            "Mean Squared Error: 7.773074152989437\n",
            "R-squared: -0.3891363173617579\n",
            "Elapsed time: 29.37 seconds\n",
            "9/9 [==============================] - 0s 3ms/step\n",
            "Mean Squared Error: 79.08137566774442\n",
            "R-squared: -13.132736778889893\n",
            "Elapsed time: 36.24 seconds\n",
            "9/9 [==============================] - 0s 3ms/step\n",
            "Mean Squared Error: 80.32409729216036\n",
            "R-squared: -13.354825196788628\n",
            "Elapsed time: 31.7 seconds\n",
            "9/9 [==============================] - 0s 3ms/step\n",
            "Mean Squared Error: 71.08688550031563\n",
            "R-squared: -11.704030914030005\n",
            "Elapsed time: 31.3 seconds\n",
            "9/9 [==============================] - 0s 3ms/step\n",
            "Mean Squared Error: 96.12652932236486\n",
            "R-squared: -16.178898633339518\n",
            "Elapsed time: 29.38 seconds\n",
            "9/9 [==============================] - 0s 3ms/step\n",
            "Mean Squared Error: 83.82320220441613\n",
            "R-squared: -13.980154843232826\n",
            "Elapsed time: 44.6 seconds\n",
            "9/9 [==============================] - 0s 4ms/step\n",
            "Mean Squared Error: 29.054871757093117\n",
            "R-squared: -4.1924343907801305\n",
            "Elapsed time: 30.65 seconds\n",
            "9/9 [==============================] - 0s 3ms/step\n",
            "Mean Squared Error: 78.95221786278756\n",
            "R-squared: -13.109654817492768\n",
            "Elapsed time: 33.68 seconds\n",
            "9/9 [==============================] - 0s 3ms/step\n",
            "Mean Squared Error: 47.92811463680031\n",
            "R-squared: -7.565296477848669\n",
            "Elapsed time: 30.92 seconds\n",
            "9/9 [==============================] - 0s 3ms/step\n",
            "Mean Squared Error: 91.95167697740452\n",
            "R-squared: -15.432805273381422\n",
            "Elapsed time: 38.29 seconds\n",
            "9/9 [==============================] - 0s 3ms/step\n",
            "Mean Squared Error: 21.966803327701605\n",
            "R-squared: -2.9257163482890145\n",
            "Elapsed time: 34.91 seconds\n",
            "9/9 [==============================] - 0s 3ms/step\n",
            "Mean Squared Error: 7.727493259339996\n",
            "R-squared: -0.38099049583734956\n",
            "Elapsed time: 44.84 seconds\n",
            "9/9 [==============================] - 0s 4ms/step\n",
            "Mean Squared Error: 17.24144414193886\n",
            "R-squared: -2.081241185911011\n",
            "Elapsed time: 30.76 seconds\n",
            "9/9 [==============================] - 0s 4ms/step\n",
            "Mean Squared Error: 34.365288048402\n",
            "R-squared: -5.141465878885725\n",
            "Elapsed time: 41.74 seconds\n",
            "9/9 [==============================] - 0s 3ms/step\n",
            "Mean Squared Error: 60.56699701060381\n",
            "R-squared: -9.824007789583877\n",
            "Elapsed time: 34.14 seconds\n",
            "5/5 [==============================] - 0s 4ms/step\n",
            "Mean Squared Error: 14.696414033745135\n",
            "R-squared: -0.8121535937837798\n",
            "Elapsed time: 22.16 seconds\n",
            "5/5 [==============================] - 0s 4ms/step\n",
            "Mean Squared Error: 33.69937683441745\n",
            "R-squared: -3.155329776266596\n",
            "Elapsed time: 22.45 seconds\n",
            "5/5 [==============================] - 0s 4ms/step\n",
            "Mean Squared Error: 20.333647288519668\n",
            "R-squared: -1.5072573434590906\n",
            "Elapsed time: 25.31 seconds\n",
            "5/5 [==============================] - 0s 4ms/step\n",
            "Mean Squared Error: 34.8995809276154\n",
            "R-squared: -3.303321943319638\n",
            "Elapsed time: 21.47 seconds\n",
            "5/5 [==============================] - 0s 4ms/step\n",
            "Mean Squared Error: 13.794168436794587\n",
            "R-squared: -0.7009014477000273\n",
            "Elapsed time: 21.07 seconds\n",
            "5/5 [==============================] - 0s 4ms/step\n",
            "Mean Squared Error: 8.898910608196031\n",
            "R-squared: -0.09728759698623901\n",
            "Elapsed time: 18.93 seconds\n",
            "5/5 [==============================] - 0s 5ms/step\n",
            "Mean Squared Error: 9.271996697371339\n",
            "R-squared: -0.1432912884811417\n",
            "Elapsed time: 34.7 seconds\n",
            "5/5 [==============================] - 0s 3ms/step\n",
            "Mean Squared Error: 23.585137661442445\n",
            "R-squared: -1.908185076640533\n",
            "Elapsed time: 22.02 seconds\n",
            "5/5 [==============================] - 0s 5ms/step\n",
            "Mean Squared Error: 8.813330941788678\n",
            "R-squared: -0.0867351248199808\n",
            "Elapsed time: 24.05 seconds\n",
            "5/5 [==============================] - 0s 4ms/step\n",
            "Mean Squared Error: 58.17516559431052\n",
            "R-squared: -6.173337329679953\n",
            "Elapsed time: 22.96 seconds\n",
            "5/5 [==============================] - 0s 4ms/step\n",
            "Mean Squared Error: 8.578245351790324\n",
            "R-squared: -0.05774770001117302\n",
            "Elapsed time: 29.48 seconds\n",
            "5/5 [==============================] - 0s 4ms/step\n",
            "Mean Squared Error: 9.1285177088315\n",
            "R-squared: -0.1255994920934027\n",
            "Elapsed time: 24.89 seconds\n",
            "5/5 [==============================] - 0s 4ms/step\n",
            "Mean Squared Error: 9.61423641814596\n",
            "R-squared: -0.1854914427849903\n",
            "Elapsed time: 25.12 seconds\n",
            "5/5 [==============================] - 0s 4ms/step\n",
            "Mean Squared Error: 41.914748776413745\n",
            "R-squared: -4.168333067734507\n",
            "Elapsed time: 20.71 seconds\n",
            "5/5 [==============================] - 0s 4ms/step\n",
            "Mean Squared Error: 9.382503460686983\n",
            "R-squared: -0.1569174171285821\n",
            "Elapsed time: 31.77 seconds\n",
            "5/5 [==============================] - 0s 5ms/step\n",
            "Mean Squared Error: 39.83943361800663\n",
            "R-squared: -3.9124346006726727\n",
            "Elapsed time: 33.86 seconds\n",
            "9/9 [==============================] - 0s 3ms/step\n",
            "Mean Squared Error: 17.87447889035191\n",
            "R-squared: -2.1795320458660705\n",
            "Elapsed time: 36.58 seconds\n",
            "9/9 [==============================] - 0s 3ms/step\n",
            "Mean Squared Error: 9.508363409829098\n",
            "R-squared: -0.6913581845236547\n",
            "Elapsed time: 33.46 seconds\n",
            "9/9 [==============================] - 0s 3ms/step\n",
            "Mean Squared Error: 31.619808816614782\n",
            "R-squared: -4.624566513704158\n",
            "Elapsed time: 41.1 seconds\n",
            "9/9 [==============================] - 0s 3ms/step\n",
            "Mean Squared Error: 8.87446950757571\n",
            "R-squared: -0.5786004371086162\n",
            "Elapsed time: 33.82 seconds\n",
            "9/9 [==============================] - 0s 3ms/step\n",
            "Mean Squared Error: 7.909532021373557\n",
            "R-squared: -0.4069562913711291\n",
            "Elapsed time: 36.42 seconds\n",
            "9/9 [==============================] - 0s 5ms/step\n",
            "Mean Squared Error: 90.29158137439686\n",
            "R-squared: -15.061166214293294\n",
            "Elapsed time: 39.5 seconds\n",
            "9/9 [==============================] - 0s 11ms/step\n",
            "Mean Squared Error: 14.851383734223399\n",
            "R-squared: -1.641780540740997\n",
            "Elapsed time: 73.52 seconds\n",
            "9/9 [==============================] - 0s 4ms/step\n",
            "Mean Squared Error: 17.735682509596156\n",
            "R-squared: -2.1548427923683726\n",
            "Elapsed time: 49.52 seconds\n",
            "9/9 [==============================] - 0s 5ms/step\n",
            "Mean Squared Error: 46.420979519760856\n",
            "R-squared: -7.2574151050210745\n",
            "Elapsed time: 70.09 seconds\n",
            "9/9 [==============================] - 0s 5ms/step\n",
            "Mean Squared Error: 45.54229269608516\n",
            "R-squared: -7.101113322390356\n",
            "Elapsed time: 53.28 seconds\n",
            "9/9 [==============================] - 0s 5ms/step\n",
            "Mean Squared Error: 25.67372787054924\n",
            "R-squared: -3.56687106807448\n",
            "Elapsed time: 72.16 seconds\n",
            "9/9 [==============================] - 0s 5ms/step\n",
            "Mean Squared Error: 28.37424294348362\n",
            "R-squared: -4.04724167173856\n",
            "Elapsed time: 72.05 seconds\n",
            "9/9 [==============================] - 0s 9ms/step\n",
            "Mean Squared Error: 15.767089038701462\n",
            "R-squared: -1.8046672116206217\n",
            "Elapsed time: 73.58 seconds\n",
            "9/9 [==============================] - 0s 4ms/step\n",
            "Mean Squared Error: 17.214235421975978\n",
            "R-squared: -2.0620872085282858\n",
            "Elapsed time: 47.19 seconds\n",
            "9/9 [==============================] - 0s 8ms/step\n",
            "Mean Squared Error: 16.83226848874879\n",
            "R-squared: -1.99414250859566\n",
            "Elapsed time: 66.49 seconds\n",
            "9/9 [==============================] - 0s 4ms/step\n",
            "Mean Squared Error: 68.85796361507849\n",
            "R-squared: -11.248530615647592\n",
            "Elapsed time: 40.86 seconds\n",
            "5/5 [==============================] - 0s 4ms/step\n",
            "Mean Squared Error: 13.040683781005548\n",
            "R-squared: -0.6065481625441631\n",
            "Elapsed time: 27.64 seconds\n",
            "5/5 [==============================] - 0s 5ms/step\n",
            "Mean Squared Error: 9.572187432685464\n",
            "R-squared: -0.17924645591884558\n",
            "Elapsed time: 22.73 seconds\n",
            "5/5 [==============================] - 0s 7ms/step\n",
            "Mean Squared Error: 6.696067546676433\n",
            "R-squared: 0.17507738136756446\n",
            "Elapsed time: 37.97 seconds\n",
            "5/5 [==============================] - 0s 6ms/step\n",
            "Mean Squared Error: 9.456527249072407\n",
            "R-squared: -0.16499768962840577\n",
            "Elapsed time: 41.07 seconds\n",
            "5/5 [==============================] - 0s 5ms/step\n",
            "Mean Squared Error: 15.025811849497844\n",
            "R-squared: -0.8511061860656282\n",
            "Elapsed time: 35.29 seconds\n",
            "5/5 [==============================] - 0s 5ms/step\n",
            "Mean Squared Error: 36.28392240113495\n",
            "R-squared: -3.470000947982772\n",
            "Elapsed time: 24.84 seconds\n",
            "5/5 [==============================] - 0s 4ms/step\n",
            "Mean Squared Error: 9.142183589463881\n",
            "R-squared: -0.12627209538563644\n",
            "Elapsed time: 33.29 seconds\n",
            "5/5 [==============================] - 0s 4ms/step\n",
            "Mean Squared Error: 9.13091573641708\n",
            "R-squared: -0.12488395125821539\n",
            "Elapsed time: 24.84 seconds\n",
            "5/5 [==============================] - 0s 5ms/step\n",
            "Mean Squared Error: 9.7381496076199\n",
            "R-squared: -0.1996921803662941\n",
            "Elapsed time: 31.33 seconds\n",
            "5/5 [==============================] - 0s 5ms/step\n",
            "Mean Squared Error: 15.229190721629534\n",
            "R-squared: -0.8761614637497159\n",
            "Elapsed time: 24.95 seconds\n",
            "5/5 [==============================] - 0s 6ms/step\n",
            "Mean Squared Error: 10.40179699828781\n",
            "R-squared: -0.2814502778678769\n",
            "Elapsed time: 38.51 seconds\n",
            "5/5 [==============================] - 0s 5ms/step\n",
            "Mean Squared Error: 12.647260507735012\n",
            "R-squared: -0.5580803484794201\n",
            "Elapsed time: 52.81 seconds\n",
            "5/5 [==============================] - 0s 5ms/step\n",
            "Mean Squared Error: 8.509507904611812\n",
            "R-squared: -0.048329559851883364\n",
            "Elapsed time: 30.08 seconds\n",
            "5/5 [==============================] - 0s 5ms/step\n",
            "Mean Squared Error: 10.053890618344735\n",
            "R-squared: -0.23858992140029267\n",
            "Elapsed time: 25.71 seconds\n",
            "5/5 [==============================] - 0s 6ms/step\n",
            "Mean Squared Error: 23.002653431521807\n",
            "R-squared: -1.8338138723889843\n",
            "Elapsed time: 38.41 seconds\n",
            "5/5 [==============================] - 0s 4ms/step\n",
            "Mean Squared Error: 21.486881002616393\n",
            "R-squared: -1.6470781573461855\n",
            "Elapsed time: 31.25 seconds\n"
          ]
        }
      ],
      "source": [
        "import itertools\n",
        "\n",
        "scalers = [MinMaxScaler(feature_range=(0, 1)), StandardScaler()]\n",
        "sequence_lengths = [10, 20]\n",
        "split_ratios = [0.8, 0.9]\n",
        "num_rnn_unitses = [64, 128]\n",
        "dense_layer_sizes = [32, 64]\n",
        "num_epochses =[70, 100]\n",
        "batch_sizes = [30, 50]\n",
        "\n",
        "# Create a list of all the parameter combinations\n",
        "param_combos = list(itertools.product(scalers, sequence_lengths, split_ratios, num_rnn_unitses, dense_layer_sizes, num_epochses, batch_sizes))\n",
        "\n",
        "# Loop through the parameter combinations and run the implementRNN function\n",
        "for params in param_combos:\n",
        "    scaler, sequence_length, split_ratio, num_rnn_units, dense_layer_size, num_epochs, batch_size = params\n",
        "    prediction2rnn = implementRNN(series, scaler, sequence_length, split_ratio, num_rnn_units, dense_layer_size, num_epochs, batch_size)\n"
      ],
      "id": "z99DnMd-k6Nv"
    },
    {
      "cell_type": "code",
      "execution_count": null,
      "metadata": {
        "id": "ldx2GU3qpGZG"
      },
      "outputs": [],
      "source": [
        "import pandas as pd\n",
        "from sklearn.metrics import mean_absolute_error, mean_squared_error, mean_absolute_percentage_error\n",
        "# Create an empty data frame with the column names\n",
        "df = pd.DataFrame(columns=['TTR', 'WS', 'RT', 'BS', 'LR', 'NORU', 'DL', 'RMSE', 'MAE', 'MAPE', 'Execution time'])\n",
        "\n",
        "# Loop through the parameter combinations and run the implementRNN function\n",
        "for params in param_combos:\n",
        "    scaler, sequence_length, split_ratio, num_rnn_units, dense_layer_size, num_epochs, batch_size = params\n",
        "    prediction2rnn, elapsed = implementRNN(series, scaler, sequence_length, split_ratio, num_rnn_units, dense_layer_size, num_epochs, batch_size) # Assume the function returns the predictions and the execution time\n",
        "    # Calculate the metrics from the predictions and the actual values\n",
        "    rmse = np.sqrt(mean_squared_error(y_test, prediction2rnn))\n",
        "    mae = mean_absolute_error(y_test, prediction2rnn)\n",
        "    mape = mean_absolute_percentage_error(y_test, prediction2rnn)\n",
        "    # Append a row to the data frame with the parameter values and the metrics\n",
        "    df = df.append({'TTR': split_ratio, 'WS': sequence_length, 'RT': scaler, 'BS': batch_size, 'LR': num_epochs, 'NORU': num_rnn_units, 'DL': dense_layer_size, 'RMSE': rmse, 'MAE': mae, 'MAPE': mape, 'Execution time': elapsed}, ignore_index=True)\n"
      ],
      "id": "ldx2GU3qpGZG"
    },
    {
      "cell_type": "code",
      "execution_count": null,
      "metadata": {
        "id": "dfc2394b"
      },
      "outputs": [],
      "source": [
        "series = set_window(14, df_daily['total_energy_consumption'])\n",
        "scaler = MinMaxScaler(feature_range=(0, 1))\n",
        "sequence_length = 50\n",
        "split_ratio = 0.95\n",
        "num_rnn_units = 128\n",
        "dense_layer_size = 80\n",
        "num_epochs = 100\n",
        "batch_size = 100\n",
        "prediction2rnn = implementRNN (series, scaler,sequence_length, split_ratio, num_rnn_units, dense_layer_size, num_epochs, batch_size)"
      ],
      "id": "dfc2394b"
    },
    {
      "cell_type": "code",
      "execution_count": null,
      "metadata": {
        "id": "kKwCDrj-X7cj"
      },
      "outputs": [],
      "source": [],
      "id": "kKwCDrj-X7cj"
    },
    {
      "cell_type": "code",
      "execution_count": null,
      "metadata": {
        "id": "d3785a4d"
      },
      "outputs": [],
      "source": [
        "# Import the necessary libraries\n",
        "import numpy as np\n",
        "import matplotlib.pyplot as plt\n",
        "from sklearn.metrics import mean_squared_error\n",
        "\n",
        "# Define the parameter grid as a dictionary\n",
        "param_grid = {\n",
        "    'num_rnn_units': [64, 128, 256],\n",
        "    'dense_layer_size': [40, 80, 120],\n",
        "    'num_epochs': [50, 100, 150],\n",
        "    'batch_size': [50, 100, 200]\n",
        "}\n",
        "\n",
        "# Initialize the best score and the best parameters\n",
        "best_score = np.inf\n",
        "best_params = None\n",
        "\n",
        "# Loop through all the possible combinations of the parameter values\n",
        "for num_rnn_units in param_grid['num_rnn_units']:\n",
        "    for dense_layer_size in param_grid['dense_layer_size']:\n",
        "        for num_epochs in param_grid['num_epochs']:\n",
        "            for batch_size in param_grid['batch_size']:\n",
        "                # Create a dictionary of the current parameter values\n",
        "                params = {\n",
        "                    'num_rnn_units': num_rnn_units,\n",
        "                    'dense_layer_size': dense_layer_size,\n",
        "                    'num_epochs': num_epochs,\n",
        "                    'batch_size': batch_size\n",
        "                }\n",
        "                # Call the implementRNN function with the current parameters\n",
        "                predictions = implementRNN(series, scaler,sequence_length, split_ratio, **params)\n",
        "                # Calculate the mean squared error of the predictions\n",
        "                mse = mean_squared_error(validation_target[:len(predictions)], predictions)\n",
        "                # Print the current parameters and the mean squared error\n",
        "                print('Parameters:', params)\n",
        "                print('Mean Squared Error:'\n",
        "                      , mse)\n",
        "                # If the mean squared error is lower than the best score, update the best score and the best parameters\n",
        "                if mse < best_score:\n",
        "                    best_score = mse\n",
        "                    best_params = params\n",
        "\n",
        "# Print the best score and the best parameters\n",
        "print('Best Score:', best_score)\n",
        "print('Best Parameters:', best_params)"
      ],
      "id": "d3785a4d"
    },
    {
      "cell_type": "markdown",
      "metadata": {
        "id": "982271c3"
      },
      "source": [
        "LSTM"
      ],
      "id": "982271c3"
    },
    {
      "cell_type": "code",
      "execution_count": null,
      "metadata": {
        "id": "ae89754d"
      },
      "outputs": [],
      "source": [
        "import numpy as np\n",
        "from sklearn.preprocessing import StandardScaler\n",
        "from tensorflow.keras.models import Sequential\n",
        "from tensorflow.keras.layers import LSTM, Dense, SimpleRNN\n",
        "\n",
        "# Assuming you have a 1D time series data as a NumPy array\n",
        "# Replace this with your actual data\n",
        "data = series\n",
        "\n",
        "# Reshape the data to be a column vector\n",
        "data = data.reshape(-1, 1)\n",
        "\n",
        "# Normalize the data using Min-Max scaling\n",
        "scaler = StandardScaler()\n",
        "data_normalized = scaler.fit_transform(data)\n",
        "\n",
        "# Define the sequence length (number of time steps for each input sequence)\n",
        "sequence_length = 10\n",
        "\n",
        "# Prepare sequences for input (X) and output (Y)\n",
        "X, Y = [], []\n",
        "\n",
        "for i in range(len(data_normalized) - sequence_length):\n",
        "    # Input sequence\n",
        "    x_sequence = data_normalized[i:i + sequence_length]\n",
        "    # Output sequence (target)\n",
        "    y_sequence = data_normalized[i + sequence_length]\n",
        "\n",
        "    X.append(x_sequence)\n",
        "    Y.append(y_sequence)\n",
        "\n",
        "# Convert the lists to NumPy arrays\n",
        "X = np.array(X)\n",
        "Y = np.array(Y)\n",
        "\n",
        "# Split the data into training and testing sets (80% training, 20% testing)\n",
        "split_ratio = 0.8\n",
        "split_index = int(split_ratio * len(X))\n",
        "\n",
        "X_train, X_test = X[:split_index], X[split_index:]\n",
        "Y_train, Y_test = Y[:split_index], Y[split_index:]\n",
        "\n",
        "# Create the LSTM model\n",
        "model = Sequential()\n",
        "model.add(LSTM(units=100, input_shape=(sequence_length, 1)))\n",
        "model.add(Dense(units=2))\n",
        "\n",
        "# Compile the model\n",
        "model.compile(optimizer='adam', loss='mean_squared_error')\n",
        "\n",
        "# Train the model on the training set\n",
        "model.fit(X_train, Y_train, epochs=100, batch_size=50, verbose=0)\n",
        "\n",
        "# Use the trained model to make predictions on the test set\n",
        "predictions = model.predict(X_test)\n",
        "\n",
        "# Inference using predicted values as input for the next prediction\n",
        "validation_target = Y_test\n",
        "validation_predictions = []\n",
        "\n",
        "last_x = X_test[0]\n",
        "\n",
        "while len(validation_predictions) < len(validation_target):\n",
        "    p = model.predict(last_x.reshape(1, sequence_length, 1), verbose=0)[0, 0]\n",
        "    validation_predictions.append(p)\n",
        "    last_x = np.roll(last_x, -1, axis=0)\n",
        "    last_x[-1] = p\n",
        "\n",
        "# Denormalize the predictions for better interpretation\n",
        "validation_predictions = scaler.inverse_transform(np.array(validation_predictions).reshape(-1, 1))\n",
        "validation_target = scaler.inverse_transform(np.array(Y_test).reshape(-1, 1))\n",
        "\n",
        "plt.plot(validation_target, label=\"targets\")\n",
        "plt.plot(validation_predictions, label=\"predictions\")\n",
        "plt.title(\"LSTM Predictions\")\n",
        "plt.legend()\n",
        "plt.show\n",
        "\n",
        "# Evaluate the model\n",
        "mse = mean_squared_error(validation_target, validation_predictions)\n",
        "r2 = r2_score(validation_target, validation_predictions)\n",
        "\n",
        "print(f'Mean Squared Error: {mse}')\n",
        "print(f'R-squared: {r2}')"
      ],
      "id": "ae89754d"
    },
    {
      "cell_type": "code",
      "execution_count": null,
      "metadata": {
        "id": "3e04be36"
      },
      "outputs": [],
      "source": [
        "def implementLSTM (series, scaler,sequence_length, split_ratio, num_rnn_units, dense_layer_size, num_epochs, batch_size):\n",
        "\n",
        "    data = series\n",
        "    data = data.reshape(-1, 1)\n",
        "    data_normalized = scaler.fit_transform(data)\n",
        "    X, Y = [], []\n",
        "    for i in range(len(data_normalized) - sequence_length):\n",
        "        x_sequence = data_normalized[i:i + sequence_length]\n",
        "        y_sequence = data_normalized[i + sequence_length]\n",
        "        X.append(x_sequence)\n",
        "        Y.append(y_sequence)\n",
        "    X = np.array(X)\n",
        "    Y = np.array(Y)\n",
        "\n",
        "    split_index = int(split_ratio * len(X))\n",
        "    X_train, X_test = X[:split_index], X[split_index:]\n",
        "    Y_train, Y_test = Y[:split_index], Y[split_index:]\n",
        "\n",
        "    # Create the RNN model\n",
        "    model = Sequential()\n",
        "    model.add(LSTM(units=num_rnn_units, input_shape=(sequence_length, 1)))\n",
        "    model.add(Dense(units=dense_layer_size))\n",
        "    model.compile(optimizer='adam', loss='mean_squared_error')\n",
        "    model.fit(X_train, Y_train, epochs=num_epochs, batch_size=batch_size, verbose=0)\n",
        "    predictions = model.predict(X_test)\n",
        "\n",
        "    # Inference using predicted values as input for the next prediction\n",
        "    validation_target = Y_test\n",
        "    validation_predictions = []\n",
        "    last_x = X_test[0]\n",
        "\n",
        "    while len(validation_predictions) < len(validation_target):\n",
        "        p = model.predict(last_x.reshape(1, sequence_length, 1), verbose=0)[0, 0]\n",
        "        validation_predictions.append(p)\n",
        "        last_x = np.roll(last_x, -1, axis=0)\n",
        "        last_x[-1] = p\n",
        "\n",
        "    validation_predictions = scaler.inverse_transform(np.array(validation_predictions).reshape(-1, 1))\n",
        "    validation_target = scaler.inverse_transform(np.array(Y_test).reshape(-1, 1))\n",
        "\n",
        "    plt.plot(validation_target, label=\"targets\")\n",
        "    plt.plot(validation_predictions, label=\"predictions\")\n",
        "    plt.title(\"LSTM Predictions\")\n",
        "    plt.legend()\n",
        "    plt.show\n",
        "\n",
        "    mse = mean_squared_error(validation_target, validation_predictions)\n",
        "    r2 = r2_score(validation_target, validation_predictions)\n",
        "\n",
        "    print(f'Mean Squared Error: {mse}')\n",
        "    print(f'R-squared: {r2}')\n",
        "    return validation_predictions"
      ],
      "id": "3e04be36"
    },
    {
      "cell_type": "code",
      "execution_count": null,
      "metadata": {
        "id": "2098aa00"
      },
      "outputs": [],
      "source": [
        "series = set_window(10, df_daily['total_energy_consumption'])\n",
        "scaler = MinMaxScaler(feature_range=(0, 1))\n",
        "sequence_length = 10\n",
        "split_ratio = 0.9\n",
        "num_lstm_units = 128\n",
        "dense_layer_size = 128\n",
        "num_epochs = 70\n",
        "batch_size = 50\n",
        "# learning_rate = 0.001\n",
        "implementLSTM (series, scaler,sequence_length, split_ratio, num_lstm_units, dense_layer_size, num_epochs, batch_size)"
      ],
      "id": "2098aa00"
    },
    {
      "cell_type": "code",
      "execution_count": null,
      "metadata": {
        "id": "c322c8f7"
      },
      "outputs": [],
      "source": [
        "# Import the necessary libraries\n",
        "import numpy as np\n",
        "import matplotlib.pyplot as plt\n",
        "from sklearn.metrics import mean_squared_error\n",
        "\n",
        "# Define the parameter grid as a dictionary\n",
        "param_grid = {\n",
        "    'num_lstm_units': [64, 128, 256],\n",
        "    'dense_layer_size': [40, 80, 120],\n",
        "    'num_epochs': [50, 100, 150],\n",
        "    'batch_size': [50, 100, 200]\n",
        "}\n",
        "\n",
        "# Initialize the best score and the best parameters\n",
        "best_score = np.inf\n",
        "best_params = None\n",
        "\n",
        "# Loop through all the possible combinations of the parameter values\n",
        "for num_lstm_units in param_grid['num_lstm_units']:\n",
        "    for dense_layer_size in param_grid['dense_layer_size']:\n",
        "        for num_epochs in param_grid['num_epochs']:\n",
        "            for batch_size in param_grid['batch_size']:\n",
        "                # Create a dictionary of the current parameter values\n",
        "                params = {\n",
        "                    'num_lstm_units': num_lstm_units,\n",
        "                    'dense_layer_size': dense_layer_size,\n",
        "                    'num_epochs': num_epochs,\n",
        "                    'batch_size': batch_size\n",
        "                }\n",
        "                # Call the implementlstm function with the current parameters\n",
        "                predictions = implementLSTM(series, scaler,sequence_length, split_ratio, **params)\n",
        "                # Calculate the mean squared error of the predictions\n",
        "                mse = mean_squared_error(validation_target[:len(predictions)], predictions)\n",
        "                # Print the current parameters and the mean squared error\n",
        "                print('Parameters:', params)\n",
        "                print('Mean Squared Error:'\n",
        "                      , mse)\n",
        "                # If the mean squared error is lower than the best score, update the best score and the best parameters\n",
        "                if mse < best_score:\n",
        "                    best_score = mse\n",
        "                    best_params = params\n",
        "\n",
        "# Print the best score and the best parameters\n",
        "print('Best Score:', best_score)\n",
        "print('Best Parameters:', best_params)"
      ],
      "id": "c322c8f7"
    },
    {
      "cell_type": "code",
      "execution_count": null,
      "metadata": {
        "id": "fb65edf8"
      },
      "outputs": [],
      "source": [],
      "id": "fb65edf8"
    },
    {
      "cell_type": "code",
      "execution_count": null,
      "metadata": {
        "id": "dbf74233"
      },
      "outputs": [],
      "source": [],
      "id": "dbf74233"
    },
    {
      "cell_type": "code",
      "execution_count": null,
      "metadata": {
        "id": "52525ee6"
      },
      "outputs": [],
      "source": [],
      "id": "52525ee6"
    },
    {
      "cell_type": "code",
      "execution_count": null,
      "metadata": {
        "id": "bed86653"
      },
      "outputs": [],
      "source": [],
      "id": "bed86653"
    },
    {
      "cell_type": "code",
      "execution_count": null,
      "metadata": {
        "id": "afe0ee4f"
      },
      "outputs": [],
      "source": [],
      "id": "afe0ee4f"
    },
    {
      "cell_type": "code",
      "execution_count": null,
      "metadata": {
        "id": "6158d289"
      },
      "outputs": [],
      "source": [],
      "id": "6158d289"
    },
    {
      "cell_type": "code",
      "execution_count": null,
      "metadata": {
        "id": "6cd03bcb"
      },
      "outputs": [],
      "source": [],
      "id": "6cd03bcb"
    },
    {
      "cell_type": "code",
      "execution_count": null,
      "metadata": {
        "id": "7f96f895"
      },
      "outputs": [],
      "source": [],
      "id": "7f96f895"
    },
    {
      "cell_type": "code",
      "execution_count": null,
      "metadata": {
        "id": "61827d19"
      },
      "outputs": [],
      "source": [],
      "id": "61827d19"
    },
    {
      "cell_type": "code",
      "execution_count": null,
      "metadata": {
        "id": "868f49e2"
      },
      "outputs": [],
      "source": [],
      "id": "868f49e2"
    },
    {
      "cell_type": "code",
      "execution_count": null,
      "metadata": {
        "id": "18fa313c"
      },
      "outputs": [],
      "source": [],
      "id": "18fa313c"
    },
    {
      "cell_type": "code",
      "execution_count": null,
      "metadata": {
        "id": "41cb4f9e"
      },
      "outputs": [],
      "source": [],
      "id": "41cb4f9e"
    },
    {
      "cell_type": "code",
      "execution_count": null,
      "metadata": {
        "id": "e79156aa"
      },
      "outputs": [],
      "source": [],
      "id": "e79156aa"
    },
    {
      "cell_type": "code",
      "execution_count": null,
      "metadata": {
        "id": "c59b3b08"
      },
      "outputs": [],
      "source": [],
      "id": "c59b3b08"
    },
    {
      "cell_type": "code",
      "execution_count": null,
      "metadata": {
        "id": "b6601e40"
      },
      "outputs": [],
      "source": [],
      "id": "b6601e40"
    },
    {
      "cell_type": "code",
      "execution_count": null,
      "metadata": {
        "id": "8e01927a"
      },
      "outputs": [],
      "source": [],
      "id": "8e01927a"
    },
    {
      "cell_type": "code",
      "execution_count": null,
      "metadata": {
        "id": "77706a6e"
      },
      "outputs": [],
      "source": [],
      "id": "77706a6e"
    },
    {
      "cell_type": "code",
      "execution_count": null,
      "metadata": {
        "id": "14a3bcb8"
      },
      "outputs": [],
      "source": [],
      "id": "14a3bcb8"
    },
    {
      "cell_type": "code",
      "execution_count": null,
      "metadata": {
        "id": "04adb390"
      },
      "outputs": [],
      "source": [],
      "id": "04adb390"
    },
    {
      "cell_type": "code",
      "execution_count": null,
      "metadata": {
        "id": "be98b374"
      },
      "outputs": [],
      "source": [],
      "id": "be98b374"
    },
    {
      "cell_type": "code",
      "execution_count": null,
      "metadata": {
        "id": "129d7677"
      },
      "outputs": [],
      "source": [],
      "id": "129d7677"
    },
    {
      "cell_type": "code",
      "execution_count": null,
      "metadata": {
        "id": "bb72c8dc"
      },
      "outputs": [],
      "source": [],
      "id": "bb72c8dc"
    },
    {
      "cell_type": "code",
      "execution_count": null,
      "metadata": {
        "id": "51fd7d65"
      },
      "outputs": [],
      "source": [],
      "id": "51fd7d65"
    },
    {
      "cell_type": "code",
      "execution_count": null,
      "metadata": {
        "id": "cc1d601a"
      },
      "outputs": [],
      "source": [],
      "id": "cc1d601a"
    },
    {
      "cell_type": "code",
      "execution_count": null,
      "metadata": {
        "id": "e6a98e16"
      },
      "outputs": [],
      "source": [],
      "id": "e6a98e16"
    },
    {
      "cell_type": "code",
      "execution_count": null,
      "metadata": {
        "id": "2485c0d5"
      },
      "outputs": [],
      "source": [],
      "id": "2485c0d5"
    },
    {
      "cell_type": "code",
      "execution_count": null,
      "metadata": {
        "id": "7cc71b90"
      },
      "outputs": [],
      "source": [],
      "id": "7cc71b90"
    },
    {
      "cell_type": "code",
      "execution_count": null,
      "metadata": {
        "id": "843d610d"
      },
      "outputs": [],
      "source": [],
      "id": "843d610d"
    },
    {
      "cell_type": "code",
      "execution_count": null,
      "metadata": {
        "id": "73567169"
      },
      "outputs": [],
      "source": [],
      "id": "73567169"
    },
    {
      "cell_type": "code",
      "execution_count": null,
      "metadata": {
        "id": "a2a731b3"
      },
      "outputs": [],
      "source": [],
      "id": "a2a731b3"
    },
    {
      "cell_type": "code",
      "execution_count": null,
      "metadata": {
        "id": "a41a5919"
      },
      "outputs": [],
      "source": [],
      "id": "a41a5919"
    },
    {
      "cell_type": "code",
      "execution_count": null,
      "metadata": {
        "id": "d150ae52"
      },
      "outputs": [],
      "source": [],
      "id": "d150ae52"
    },
    {
      "cell_type": "code",
      "execution_count": null,
      "metadata": {
        "id": "9d822f5f"
      },
      "outputs": [],
      "source": [],
      "id": "9d822f5f"
    },
    {
      "cell_type": "code",
      "execution_count": null,
      "metadata": {
        "id": "363892cc"
      },
      "outputs": [],
      "source": [],
      "id": "363892cc"
    },
    {
      "cell_type": "code",
      "execution_count": null,
      "metadata": {
        "id": "32949747"
      },
      "outputs": [],
      "source": [],
      "id": "32949747"
    },
    {
      "cell_type": "code",
      "execution_count": null,
      "metadata": {
        "id": "7fc42e68"
      },
      "outputs": [],
      "source": [],
      "id": "7fc42e68"
    },
    {
      "cell_type": "code",
      "execution_count": null,
      "metadata": {
        "id": "03d9b5cf"
      },
      "outputs": [],
      "source": [],
      "id": "03d9b5cf"
    },
    {
      "cell_type": "code",
      "execution_count": null,
      "metadata": {
        "id": "6a5b6e10"
      },
      "outputs": [],
      "source": [],
      "id": "6a5b6e10"
    },
    {
      "cell_type": "code",
      "execution_count": null,
      "metadata": {
        "id": "15e85369"
      },
      "outputs": [],
      "source": [],
      "id": "15e85369"
    },
    {
      "cell_type": "code",
      "execution_count": null,
      "metadata": {
        "id": "4eb18320"
      },
      "outputs": [],
      "source": [],
      "id": "4eb18320"
    },
    {
      "cell_type": "code",
      "execution_count": null,
      "metadata": {
        "id": "0aa39bd0"
      },
      "outputs": [],
      "source": [],
      "id": "0aa39bd0"
    },
    {
      "cell_type": "code",
      "execution_count": null,
      "metadata": {
        "id": "ebbf86b3"
      },
      "outputs": [],
      "source": [],
      "id": "ebbf86b3"
    },
    {
      "cell_type": "code",
      "execution_count": null,
      "metadata": {
        "id": "1d08f91f"
      },
      "outputs": [],
      "source": [],
      "id": "1d08f91f"
    },
    {
      "cell_type": "code",
      "execution_count": null,
      "metadata": {
        "id": "8c008667"
      },
      "outputs": [],
      "source": [],
      "id": "8c008667"
    },
    {
      "cell_type": "code",
      "execution_count": null,
      "metadata": {
        "id": "d1fcec28"
      },
      "outputs": [],
      "source": [],
      "id": "d1fcec28"
    },
    {
      "cell_type": "code",
      "execution_count": null,
      "metadata": {
        "id": "e2fb81c4"
      },
      "outputs": [],
      "source": [],
      "id": "e2fb81c4"
    },
    {
      "cell_type": "code",
      "execution_count": null,
      "metadata": {
        "id": "0ff35fe5"
      },
      "outputs": [],
      "source": [],
      "id": "0ff35fe5"
    },
    {
      "cell_type": "code",
      "execution_count": null,
      "metadata": {
        "id": "4de17362"
      },
      "outputs": [],
      "source": [],
      "id": "4de17362"
    },
    {
      "cell_type": "code",
      "execution_count": null,
      "metadata": {
        "id": "c260171a"
      },
      "outputs": [],
      "source": [],
      "id": "c260171a"
    },
    {
      "cell_type": "code",
      "execution_count": null,
      "metadata": {
        "id": "269629be"
      },
      "outputs": [],
      "source": [],
      "id": "269629be"
    }
  ],
  "metadata": {
    "colab": {
      "provenance": []
    },
    "kernelspec": {
      "display_name": "Python 3 (ipykernel)",
      "language": "python",
      "name": "python3"
    },
    "language_info": {
      "codemirror_mode": {
        "name": "ipython",
        "version": 3
      },
      "file_extension": ".py",
      "mimetype": "text/x-python",
      "name": "python",
      "nbconvert_exporter": "python",
      "pygments_lexer": "ipython3",
      "version": "3.11.4"
    }
  },
  "nbformat": 4,
  "nbformat_minor": 5
}

NameError: name 'null' is not defined